## Define the Pairwise Dynamics of 2 Spherobots Using 
### (L, Theta1, Theta2) and (Hx, Hy, Theta_BW)

In [248]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [249]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
csfont = {'fontname':'Times New Roman'}
L = 0.1
L_norm = L/RADIUSLARGE

#Lists
#RLength
ReList=['2','7','10']
ThetaList=['0.0','22.5','45.0','67.5','90.0','112.5','135.0','157.5','180.0',
           '202.5','225.0','247.5','270.0','292.5','315.0','337.5']
HxList=['0.5','1.5','2.5','3.5','4.5','5.5','6.5','7.5','8.5','9.5','10.5','11.5','12.5']
HyList=['-13','-11','-9','-7','-5','-3','-1','1','3','5','7','9']

allData = []

#### Obtain Pos Data for all simulations
#### Limit time to first five oscillations

In [250]:
def CheckBounds(delta,val):
    global L
    if delta > L/2.0:
        delta = delta - L
    elif delta < -L/2.0:
        delta = delta + L
    newval = val + delta
    return (delta,newval)

def StoreData(cwd_POSDATA, parHx, parHy, parTheta):
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData1 = pd.read_csv(cwd_POSDATA+'pd.txt',delimiter=' ')
    file = pathlib.Path(cwd_POSDATA+'pd2.txt')
    boolPD = 0
    if file.exists ():
        boolPD = 1
        pdData2 = pd.read_csv(cwd_POSDATA+'/pd2.txt',delimiter=' ')
        pdData = pd.concat([pdData1,pdData2],ignore_index=True)
    else:
        pdData = pdData1.copy() 
    #Load pd3.txt
    file = pathlib.Path(cwd_POSDATA+'pd3.txt')
    if file.exists():
        boolPD = 1
        pdData3 = pd.read_csv(cwd_POSDATA+'/pd3.txt',delimiter=' ')
        pdData = pd.concat([pdData,pdData3],ignore_index=True)
    #Split up individual sphere data by given index
    UAdata = pdData[pdData['idx'] == 6].copy()
    LAdata = pdData[pdData['idx'] == 19].copy()
    UBdata = pdData[pdData['idx'] == 32].copy()
    LBdata = pdData[pdData['idx'] == 45].copy()
    #Sort data by time and reset indices
    UAdata = UAdata.sort_values(by=['time'])
    LAdata = LAdata.sort_values(by=['time'])
    UBdata = UBdata.sort_values(by=['time'])
    LBdata = LBdata.sort_values(by=['time'])
    UAdata = UAdata.reset_index(drop=True)
    LAdata = LAdata.reset_index(drop=True)
    UBdata = UBdata.reset_index(drop=True)
    LBdata = LBdata.reset_index(drop=True)
    #Rename columns to previous data frames
    UAdata = UAdata.rename(columns={"x":"aXU", "y":"aYU"})
    LAdata = LAdata.rename(columns={"x":"aXL", "y":"aYL"})
    UBdata = UBdata.rename(columns={"x":"bXU", "y":"bYU"})
    LBdata = LBdata.rename(columns={"x":"bXL", "y":"bYL"})
    #Combine separate dataframes to create previous dataframe used
    splitDict = {'aXU':UAdata['aXU'],'aYU':UAdata['aYU'],'aXL':LAdata['aXL'],'aYL':LAdata['aYL'],
                 'bXU':UBdata['bXU'],'bYU':UBdata['bYU'],'bXL':LBdata['bXL'],'bYL':LBdata['bYL'],'time':UAdata['time']}
    posData = pd.DataFrame(data=splitDict)
    if boolPD:
        posData = posData.drop_duplicates(subset=["time"])
    #Calculate initial positioning given Hx, Hy and Theta
    #Add initial positioning
    initDict = {'aXU':[0.0],'aYU':[1.0e-3],'aXL':[0.0],'aYL':[-4.0e-3],
                'bXU':[parHx*RADIUSSMALL - 1.0e-3*np.sin(parTheta*np.pi/180.0)],'bYU':[parHy*RADIUSSMALL + 1.0e-3*np.cos(parTheta*np.pi/180.0)],
                'bXL':[parHx*RADIUSSMALL + 4.0e-3*np.sin(parTheta*np.pi/180.0)],'bYL':[parHy*RADIUSSMALL - 4.0e-3*np.cos(parTheta*np.pi/180.0)],
                'time':[0.0]}
    initData = pd.DataFrame(data=initDict)
    posData = posData.append(initData)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    #Save only every 20 rows (Every period)
    posData = posData.iloc[::20]
    #Reindex so index corresponds to period number
    posData = posData.reset_index(drop=True)
    
    #Account for Periodic Conditions
    #Whenever a sphere crosses the boundary
    posData['adXU'] = 0.0
    posData['adYU'] = 0.0
    posData['adXL'] = 0.0
    posData['adYL'] = 0.0
    posData['bdXU'] = 0.0
    posData['bdYU'] = 0.0
    posData['bdXL'] = 0.0
    posData['bdYL'] = 0.0
    posData['aXUn'] = posData['aXU'].copy()
    posData['aYUn'] = posData['aYU'].copy()
    posData['aXLn'] = posData['aXL'].copy()
    posData['aYLn'] = posData['aYL'].copy()
    posData['bXUn'] = posData['bXU'].copy()
    posData['bYUn'] = posData['bYU'].copy()
    posData['bXLn'] = posData['bXL'].copy()
    posData['bYLn'] = posData['bYL'].copy()
    for idx in range(1,len(posData['time'])):
        #Swimmer 1
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'adXU'] = posData.loc[idx,'aXU'] - posData.loc[idx-1,'aXUn']
        posData.loc[idx,'adYU'] = posData.loc[idx,'aYU'] - posData.loc[idx-1,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXU'], posData.loc[idx,'aXUn'] = CheckBounds(posData.loc[idx,'adXU'],posData.loc[idx-1,'aXUn'])
        posData.loc[idx,'adYU'], posData.loc[idx,'aYUn'] = CheckBounds(posData.loc[idx,'adYU'],posData.loc[idx-1,'aYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'adXL'] = posData.loc[idx,'aXL'] - posData.loc[idx,'aXUn']
        posData.loc[idx,'adYL'] = posData.loc[idx,'aYL'] - posData.loc[idx,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXL'], posData.loc[idx,'aXLn'] = CheckBounds(posData.loc[idx,'adXL'],posData.loc[idx,'aXUn'])
        posData.loc[idx,'adYL'], posData.loc[idx,'aYLn'] = CheckBounds(posData.loc[idx,'adYL'],posData.loc[idx,'aYUn'])
        #Swimmer 2
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'bdXU'] = posData.loc[idx,'bXU'] - posData.loc[idx-1,'bXUn']
        posData.loc[idx,'bdYU'] = posData.loc[idx,'bYU'] - posData.loc[idx-1,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXU'], posData.loc[idx,'bXUn'] = CheckBounds(posData.loc[idx,'bdXU'],posData.loc[idx-1,'bXUn'])
        posData.loc[idx,'bdYU'], posData.loc[idx,'bYUn'] = CheckBounds(posData.loc[idx,'bdYU'],posData.loc[idx-1,'bYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'bdXL'] = posData.loc[idx,'bXL'] - posData.loc[idx,'bXUn']
        posData.loc[idx,'bdYL'] = posData.loc[idx,'bYL'] - posData.loc[idx,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXL'], posData.loc[idx,'bXLn'] = CheckBounds(posData.loc[idx,'bdXL'],posData.loc[idx,'bXUn'])
        posData.loc[idx,'bdYL'], posData.loc[idx,'bYLn'] = CheckBounds(posData.loc[idx,'bdYL'],posData.loc[idx,'bYUn'])
        
    
    #Create Swimmer A and Swimmer B dataframes
    #dict_A = {'xU':posData['aXU'],'yU':posData['aYU'],'xL':posData['aXL'],'yL':posData['aYL'],'time':posData['time']}
    #dataA  = pd.DataFrame(data=dict_A)
    #dict_B = {'xU':posData['bXU'],'yU':posData['bYU'],'xL':posData['bXL'],'yL':posData['bYL'],'time':posData['time']}
    #dataB  = pd.DataFrame(data=dict_B)
    dict_A = {'xU':posData['aXUn'],'yU':posData['aYUn'],'xL':posData['aXLn'],'yL':posData['aYLn'],'time':posData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':posData['bXUn'],'yU':posData['bYUn'],'xL':posData['bXLn'],'yL':posData['bYLn'],'time':posData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #Create a swimmer class for each
    #print('SwimA')
    swimA = Swimmer(dataA)
    #print('SwimB')
    swimB = Swimmer(dataB)
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #Calculate the three parameters (Hx, Hy, Theta_BW)
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, theta_BW = swimB.CalcHxHyTheta(swimA)

    #First, create a dataframe to contain this information
    dict_Param = {'Hx':Hx,'Hy':Hy,'ThetaBW':theta_BW,'time':posData['time']}
    parData = pd.DataFrame(data=dict_Param)
    
    return parData


#### Find Angle Between Swimmers (Account for Quadrant)

In [251]:
def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.normLSS = np.zeros((2,len(self.time)))
        self.CalcCM()
        self.CalcLabAngle()
    
    def get_theta(self):
        return self.theta
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM[:,0:10])
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmer):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmer.normX,swimmer.normY,swimmer.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmer.theta
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmer.rot_norm[:,idx] = Rotate(swimmer.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSLARGE, self.Hy/RADIUSLARGE, self.theta_BW)
    
    def Projection(self,normX,normY,CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-CM[:,idx]))


#### Create dataframe of all pos data for all time for all sims

In [252]:
#The main goal of this script is to create an H-Theta Phase Space of all
#simulations for every period elapsed.
#Example: 20s of sim time = 200 periods. 200 H-Theta Plots
#We may find that we can combine the H-Theta data after steady state has been reached
#1) For each simulation, store position data for every period (nothing in between)
#2) Calculate separation distance between large spheres (H)
#3) Calculate relative heading (Theta)
#4) Calculate deltaH and deltaTheta (change after one Period)
#5) Plot H vs Theta (Polar) for each period

ReList=['25']
HxList=['8.5']
nTheta = len(ThetaList)
nHx = len(HxList)
nHy = len(HyList)

for Re in ReList:
    count = 0
    dict_allData = {'Hx':[],'Hy':[],'ThetaBW':[],'parHx':[],'parHy':[],'parThetaBW':[],'time':[]}
    df_allData = pd.DataFrame(data=dict_allData)
    for Theta in ThetaList:
        for Hx in HxList:
            cwd_HX = cwd_PYTHON+'/../PosData/Re'+Re+'/Theta'+Theta+'/Hx'+Hx+'/'
            dirNames = [ name for name in os.listdir(cwd_HX) 
                        if os.path.isdir(os.path.join(cwd_HX, name)) ]
            print('dirHys = ',dirNames)
            for dirHy in dirNames:
                cwd_POSDATA = cwd_HX+dirHy+'/'
                HyString, HyValue = tuple(re.split('Hy',dirHy)[:2])
                strPrint = 'Theta:'+Theta+'\tHx:'+Hx+'\t'+dirHy
                print(strPrint)
                #Get All sim data and store
                simData = StoreData(cwd_POSDATA,float(Hx),float(HyValue),float(Theta))
                #Add Simulation Parameter identifiers
                simData['parThetaBW'] = np.round(180.0*simData.loc[0,'ThetaBW']/np.pi,1)
                simData['parHx'] = np.round(simData.loc[0,'Hx'],2)
                simData['parHy'] = np.round(simData.loc[0,'Hy'],1)
                dict_sim = {'parHx':simData['parHx'],'parHy':simData['parHy'],'parThetaBW':simData['parThetaBW'],
                            'Hx':simData['Hx'],'Hy':simData['Hy'],'ThetaBW':simData['ThetaBW'],
                            'time':simData['time']}
                df_sim = pd.DataFrame(data=dict_sim)
                df_allData = pd.concat([df_allData,df_sim],ignore_index=True)
                count += 1
        print('Theta ='+Theta)
    allData = df_allData.copy()
    allData = allData.sort_values(by=['parThetaBW','parHx','parHy','time'])
    allData['H'] = np.hypot(allData['Hx'],allData['Hy'])
    pathlib.Path(cwd_PYTHON+'/SimData/').mkdir(parents=True, exist_ok=True)
    allData.to_csv(cwd_PYTHON+'/SimData/allSimData_Re'+Re+'.csv',index=False,sep=' ')
    print(len(allData['time']))
    print('count = ',count)
    print('Storing Data is complete! Onto Analysis')
print('Done Storing Data for all Re')
                

dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy0', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:0.0	Hx:8.5	Hy-1


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Theta:0.0	Hx:8.5	Hy-11
Theta:0.0	Hx:8.5	Hy-13
Theta:0.0	Hx:8.5	Hy-3
Theta:0.0	Hx:8.5	Hy-5
Theta:0.0	Hx:8.5	Hy-7
Theta:0.0	Hx:8.5	Hy-9
Theta:0.0	Hx:8.5	Hy0
Theta:0.0	Hx:8.5	Hy1
Theta:0.0	Hx:8.5	Hy3
Theta:0.0	Hx:8.5	Hy5
Theta:0.0	Hx:8.5	Hy7
Theta:0.0	Hx:8.5	Hy9
Theta =0.0
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy0', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:22.5	Hx:8.5	Hy-1
Theta:22.5	Hx:8.5	Hy-11
Theta:22.5	Hx:8.5	Hy-13
Theta:22.5	Hx:8.5	Hy-3
Theta:22.5	Hx:8.5	Hy-5
Theta:22.5	Hx:8.5	Hy-7
Theta:22.5	Hx:8.5	Hy-9
Theta:22.5	Hx:8.5	Hy0
Theta:22.5	Hx:8.5	Hy1
Theta:22.5	Hx:8.5	Hy3
Theta:22.5	Hx:8.5	Hy5
Theta:22.5	Hx:8.5	Hy7
Theta:22.5	Hx:8.5	Hy9
Theta =22.5
dirHys =  ['Hy-1', 'Hy-11', 'Hy-13', 'Hy-3', 'Hy-5', 'Hy-7', 'Hy-9', 'Hy0', 'Hy1', 'Hy3', 'Hy5', 'Hy7', 'Hy9']
Theta:45.0	Hx:8.5	Hy-1
Theta:45.0	Hx:8.5	Hy-11
Theta:45.0	Hx:8.5	Hy-13
Theta:45.0	Hx:8.5	Hy-3
Theta:45.0	Hx:8.5	Hy-5
Theta:45.0	Hx:8.5	Hy-7
Theta:45.0	Hx:8.5	Hy-9
Theta:45.0	Hx:8.5	Hy0
Theta:45.0	Hx:8

#### Plot The Phase Space in 2D plane slices
##### (Hx, Hy) (H, Theta)

In [170]:
#Calculate End States with these decision trees
import matplotlib
def WhichEndStateRe2(data):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0
    #V-Shape = 1
    #In-line = 2
    #Headbutt = 3
    #L-shape = 4
    #NOTA = 5
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 54 or 306; H_bw ~ 3.8R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #In-line: Theta = 0, 360; H_bw = 4.6-4.7R
    #L-Shape: Theta = 95-100, 260-265; H_bw = 4.5-4.6R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 5
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    for idx in range(len(data['time'])):
        #print('H = ',data.loc[idx,'H'])
        #print('Theta_deg = ',data.loc[idx,'Theta_deg'])
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 4.9 and data.loc[idx,'H'] >= 4.7) and 
            ((data.loc[idx,'Theta_deg'] < 60.0 and data.loc[idx,'Theta_deg'] >= 50.0) or
            (data.loc[idx,'Theta_deg'] < 310.0 and data.loc[idx,'Theta_deg'] >= 300.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            data.loc[idx,'color_r'] = 117
            data.loc[idx,'color_g'] = 112
            data.loc[idx,'color_b'] = 179
        #In-Line
        elif((data.loc[idx,'H'] < 4.7 and data.loc[idx,'H'] >= 4.5) and 
             ((data.loc[idx,'Theta_deg'] < 5.0 and data.loc[idx,'Theta_deg'] >= 0.0) or
              (data.loc[idx,'Theta_deg'] <= 360.0 and data.loc[idx,'Theta_deg'] >= 355.0))):
            #print('idx = ',idx)
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'green'
            data.loc[idx,'color_r'] = 27
            data.loc[idx,'color_g'] = 158
            data.loc[idx,'color_b'] = 119
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.3) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            data.loc[idx,'color_r'] = 217
            data.loc[idx,'color_g'] = 95
            data.loc[idx,'color_b'] = 2
        #L-Shape (Metastable)
        elif((data.loc[idx,'H'] < 4.6 and data.loc[idx,'H'] >= 4.4) and 
             ((data.loc[idx,'Theta_deg'] < 100.0 and data.loc[idx,'Theta_deg'] >= 95.0) or
              (data.loc[idx,'Theta_deg'] < 265.0 and data.loc[idx,'Theta_deg'] >= 260.0))):
            data.loc[idx,'State'] = 4
            data.loc[idx,'color'] = 'yellow'
            data.loc[idx,'color_r'] = 230
            data.loc[idx,'color_g'] = 171
            data.loc[idx,'color_b'] = 2
    
    return data

def WhichEndStateRe7(data):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0: Red
    #V-Shape = 1: Blue
    #SingleFile = 2: green
    #Headbutt = 3: orange
    #Orbit = 4: pink
    #NOTA = 5: black
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 29 or 331; H_bw ~ 3.0R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #Orbit: Theta = 180; H_bw = 3.0-3.25R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 5
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 3.4 and data.loc[idx,'H'] >= 3.2) and 
             ((data.loc[idx,'Theta_deg'] < 35.0 and data.loc[idx,'Theta_deg'] >= 28.0) or
              (data.loc[idx,'Theta_deg'] < 332.0 and data.loc[idx,'Theta_deg'] >= 325.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            data.loc[idx,'color_r'] = 117
            data.loc[idx,'color_g'] = 112
            data.loc[idx,'color_b'] = 179
        #Headbutt
        elif((data.loc[idx,'H'] < 3.4 and data.loc[idx,'H'] >= 3.2) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) and
             (data.loc[idx,'Hx'] <= 0.1 and data.loc[idx,'Hx'] > -0.1)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            data.loc[idx,'color_r'] = 217
            data.loc[idx,'color_g'] = 95
            data.loc[idx,'color_b'] = 2
        #Orbit
        elif((data.loc[idx,'H'] < 3.4 and data.loc[idx,'H'] >= 3.2) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) ):
            data.loc[idx,'State'] = 4
            data.loc[idx,'color'] = 'pink'
            data.loc[idx,'color_r'] = 231
            data.loc[idx,'color_g'] = 41
            data.loc[idx,'color_b'] = 138
        #SingleFile
        elif((data.loc[idx,'H'] < 6.0 and data.loc[idx,'H'] >= 4.5) and 
             ((data.loc[idx,'Theta_deg'] < 45.0 and data.loc[idx,'Theta_deg'] >= 0.0) or 
              (data.loc[idx,'Theta_deg'] < 360.0 and data.loc[idx,'Theta_deg'] >= 315.0))):
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'green'
            data.loc[idx,'color_r'] = 27
            data.loc[idx,'color_g'] = 158
            data.loc[idx,'color_b'] = 119
    
    return data

def WhichEndStateRe10(data):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0: Red
    #V-Shape = 1: Blue
    #SingleFile = 2: green
    #Headbutt = 3: orange
    #Orbit = 4: pink
    #NOTA = 5: black
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 29 or 331; H_bw ~ 3.0R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #Orbit: Theta = 180; H_bw = 3.0-3.25R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 5
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 3.1 and data.loc[idx,'H'] >= 2.8) and 
             ((data.loc[idx,'Theta_deg'] < 32.0 and data.loc[idx,'Theta_deg'] >= 26.0) or
              (data.loc[idx,'Theta_deg'] < 334.0 and data.loc[idx,'Theta_deg'] >= 328.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            data.loc[idx,'color_r'] = 117
            data.loc[idx,'color_g'] = 112
            data.loc[idx,'color_b'] = 179
        #Orbit
        elif((data.loc[idx,'H'] < 3.25 and data.loc[idx,'H'] >= 3.0) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) ):
            data.loc[idx,'State'] = 4
            data.loc[idx,'color'] = 'pink'
            data.loc[idx,'color_r'] = 231
            data.loc[idx,'color_g'] = 41
            data.loc[idx,'color_b'] = 138
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.25) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) and
             (data.loc[idx,'Hx'] <= 0.1 and data.loc[idx,'Hx'] > -0.1)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            data.loc[idx,'color_r'] = 217
            data.loc[idx,'color_g'] = 95
            data.loc[idx,'color_b'] = 2
        #SingleFile
        elif((data.loc[idx,'H'] < 6.0 and data.loc[idx,'H'] >= 4.5) and 
             ((data.loc[idx,'Theta_deg'] < 10.0 and data.loc[idx,'Theta_deg'] >= 0.0) or 
              (data.loc[idx,'Theta_deg'] < 360.0 and data.loc[idx,'Theta_deg'] >= 350.0))):
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'green'
            data.loc[idx,'color_r'] = 27
            data.loc[idx,'color_g'] = 158
            data.loc[idx,'color_b'] = 119
    
    return data

def WhichEndStateRe25(data):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0: Red
    #V-Shape = 1: Blue
    #SingleFile = 2: green
    #Headbutt = 3: orange
    #Orbit = 4: pink
    #NOTA = 5: black
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 29 or 331; H_bw ~ 3.0R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #Orbit: Theta = 180; H_bw = 3.0-3.25R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 5
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['ThetaBW']/np.pi
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 2.65 and data.loc[idx,'H'] >= 2.45) and 
             ((data.loc[idx,'Theta_deg'] < 30.0 and data.loc[idx,'Theta_deg'] >= 10.0) or
              (data.loc[idx,'Theta_deg'] < 350.0 and data.loc[idx,'Theta_deg'] >= 330.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            data.loc[idx,'color_r'] = 117
            data.loc[idx,'color_g'] = 112
            data.loc[idx,'color_b'] = 179
        #Orbit
        elif((data.loc[idx,'H'] < 2.9 and data.loc[idx,'H'] >= 2.7) and 
             (data.loc[idx,'Theta_deg'] < 190.0 and data.loc[idx,'Theta_deg'] >= 170.0) ):
            data.loc[idx,'State'] = 4
            data.loc[idx,'color'] = 'pink'
            data.loc[idx,'color_r'] = 231
            data.loc[idx,'color_g'] = 41
            data.loc[idx,'color_b'] = 138
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.25) and 
             (data.loc[idx,'Theta_deg'] < 190.0 and data.loc[idx,'Theta_deg'] >= 170.0) and
             (data.loc[idx,'Hx'] <= 0.5 and data.loc[idx,'Hx'] > -0.5)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            data.loc[idx,'color_r'] = 217
            data.loc[idx,'color_g'] = 95
            data.loc[idx,'color_b'] = 2
    
    return data

### Calculate End States using decision trees in cell above

In [171]:
def DetermineEndStates(Re):
    endDict = {'parThetaBW':[],'parHx':[],
           'parHy':[],'ThetaBW':[],
           'Hx':[],'Hy':[],'H':[],
           'Nosc':[],'State':[],
           'color':[],'color_r':[],
           'color_g':[],'color_b':[]}

    endSimData = pd.DataFrame(data=endDict)
    allSimData = pd.read_csv(cwd_PYTHON+'/SimData/allSimData_Re{0}.csv'.format(Re),delimiter=' ')
    print(allSimData['time'].head())
    for Theta in ThetaList:
        print('Theta = ',Theta)
        thetaData = allSimData[allSimData['parThetaBW'] <= float(Theta)+0.01].copy()
        thetaData = thetaData[thetaData['parThetaBW'] >= float(Theta)-0.01].copy()
        for Hx in HxList:
            #print('Hx = ',float(Hx)/2.0)
            hxData = thetaData[thetaData['parHx'] <= float(Hx)/2.0 + 0.01].copy()
            hxData = hxData[hxData['parHx'] >= float(Hx)/2.0 - 0.01].copy()
            HyList = hxData['parHy'].values.tolist()
            HyList = [round(value,1) for value in HyList]
            HyList = list(set(HyList))
            for HyValue in HyList:
                HyValue2 = round(float(HyValue),1)
                #print('Hy = ',HyValue2)
                #print(HyValue2)
                hyData = hxData[hxData['parHy'] <= HyValue2+0.01].copy()
                hyData = hyData[hyData['parHy'] >= HyValue2-0.01].copy()
                hyData = hyData.sort_values(by=['time'])
                hyData = hyData.reset_index(drop=True)
                if int(Re) == 2:
                    simData = WhichEndStateRe2(hyData)
                elif int(Re) == 7:
                    simData = WhichEndStateRe7(hyData)
                elif int(Re) == 10:
                    simData = WhichEndStateRe10(hyData)
                elif int(Re) == 25:
                    simData = WhichEndStateRe25(hyData)
                #print(simData.head(20))
                lastData = simData.tail(1)
                lastIndexList = lastData.index.values
                lastIndex = lastIndexList[0]
                lastState = simData.loc[lastIndex,'State']
                if(lastIndex != 200):
                    print('LastIndex: ',lastIndex)
                #Find when Simulation is in end state
                count = 5
                boolEnd = 0
                for count in range(5,lastIndex-5):
                    State = simData.loc[count,'State']
                    if(State != 5):
                        b4State = simData.loc[count-5,'State']
                        #a4State = simData.loc[lastIndex-5,'State']
                        a4State = simData.loc[count+5,'State']
                        #Is config consistent for 10 oscillations?
                        if(State == b4State and State == a4State):
                            #If lastState != 5, is the last time state = the current one?
                            if lastState < 5 and lastState == State:
                                #Simulation is in end state!
                                print('End State! Theta: {0}, Hx: {1}, Hy:{2}, Nosc: {3}'.format(Theta,float(Hx)/2.0,HyValue2,count))
                                #Create a database with sim information
                                simDict = {'parThetaBW':[simData.loc[count,'parThetaBW']],'parHx':[simData.loc[count,'parHx']],
                                           'parHy':[simData.loc[count,'parHy']],'ThetaBW':[simData.loc[count,'ThetaBW']],
                                           'Hx':[simData.loc[count,'Hx']],'Hy':[simData.loc[count,'Hy']],'H':[simData.loc[count,'H']],
                                           'Nosc':[count],'State':[simData.loc[count,'State']],'color':[simData.loc[count,'color']],
                                           'color_r':[simData.loc[count,'color_r']],'color_g':[simData.loc[count,'color_g']],
                                           'color_b':[simData.loc[count,'color_b']]}
                                simData = pd.DataFrame(data=simDict)
                                endSimData = pd.concat([endSimData,simData],ignore_index=True)
                                boolEnd = 1
                                break
                if(boolEnd == 0):
                    print('No End State! Theta: {0}, Hx: {1}, Hy:{2}, Nosc: {3}'.format(Theta,float(Hx)/2.0,HyValue2,count))
    endSimData = endSimData.sort_values(by=['parThetaBW','parHx','parHy'])
    endSimData = endSimData.reset_index(drop=True)

    endSimData = endSimData[endSimData['color'] != 'black'].copy()
    endSimData = endSimData.reset_index(drop=True)
    endSimData['ColorString'] = 0.0
    endSimData['color_tuple'] = 's'
    for jdx in range(len(endSimData['color_r'])):
        endSimData.loc[jdx,'ColorString'] = 'rgb({0},{1},{2})'.format(endSimData.loc[jdx,'color_r'],
                                                                      endSimData.loc[jdx,'color_g'],
                                                                      endSimData.loc[jdx,'color_b'])
        endSimData = endSimData.set_value(jdx,'color_tuple', (endSimData.loc[jdx,'color_r']/255.0,endSimData.loc[jdx,'color_g']/255.0,endSimData.loc[jdx,'color_b']/255.0))

    endSimData.to_csv(cwd_PYTHON+'/SimData/allEndData_Re{0}_.csv'.format(Re),index=False,sep=' ',float_format='%.5e')

ReList = ['25']
for Re in ReList:
    print(Re)
    #if int(Re) == 7:
    DetermineEndStates(Re)
#Now that we have the end states, we can make slices of the data with the different initial parameters


25
0    0.0
1    0.1
2    0.2
3    0.3
4    0.4
Name: time, dtype: float64
Theta =  0.0
LastIndex:  141
End State! Theta: 0.0, Hx: 4.25, Hy:-0.5, Nosc: 21
LastIndex:  174
End State! Theta: 0.0, Hx: 4.25, Hy:0.0, Nosc: 23
LastIndex:  167
End State! Theta: 0.0, Hx: 4.25, Hy:0.5, Nosc: 25
LastIndex:  159
End State! Theta: 0.0, Hx: 4.25, Hy:1.5, Nosc: 26
LastIndex:  180
End State! Theta: 0.0, Hx: 4.25, Hy:2.5, Nosc: 32
LastIndex:  160
End State! Theta: 0.0, Hx: 4.25, Hy:3.5, Nosc: 42
LastIndex:  128
End State! Theta: 0.0, Hx: 4.25, Hy:4.5, Nosc: 68
LastIndex:  143
End State! Theta: 0.0, Hx: 4.25, Hy:-6.5, Nosc: 102
LastIndex:  128
End State! Theta: 0.0, Hx: 4.25, Hy:-5.5, Nosc: 84
LastIndex:  154
End State! Theta: 0.0, Hx: 4.25, Hy:-4.5, Nosc: 82
LastIndex:  182
End State! Theta: 0.0, Hx: 4.25, Hy:-3.5, Nosc: 42
LastIndex:  188
End State! Theta: 0.0, Hx: 4.25, Hy:-2.5, Nosc: 32
LastIndex:  157
End State! Theta: 0.0, Hx: 4.25, Hy:-1.5, Nosc: 23
Theta =  22.5
LastIndex:  162
End State! Theta

LastIndex:  144
End State! Theta: 202.5, Hx: 4.25, Hy:0.0, Nosc: 32
LastIndex:  150
End State! Theta: 202.5, Hx: 4.25, Hy:0.5, Nosc: 34
LastIndex:  121
End State! Theta: 202.5, Hx: 4.25, Hy:1.5, Nosc: 52
LastIndex:  122
End State! Theta: 202.5, Hx: 4.25, Hy:2.5, Nosc: 38
LastIndex:  114
End State! Theta: 202.5, Hx: 4.25, Hy:3.5, Nosc: 63
LastIndex:  108
End State! Theta: 202.5, Hx: 4.25, Hy:4.5, Nosc: 98
LastIndex:  120
End State! Theta: 202.5, Hx: 4.25, Hy:-6.5, Nosc: 33
LastIndex:  131
End State! Theta: 202.5, Hx: 4.25, Hy:-5.5, Nosc: 34
LastIndex:  127
End State! Theta: 202.5, Hx: 4.25, Hy:-4.5, Nosc: 61
LastIndex:  121
End State! Theta: 202.5, Hx: 4.25, Hy:-3.5, Nosc: 29
LastIndex:  121
End State! Theta: 202.5, Hx: 4.25, Hy:-2.5, Nosc: 21
LastIndex:  156
End State! Theta: 202.5, Hx: 4.25, Hy:-1.5, Nosc: 22
Theta =  225.0
LastIndex:  139
End State! Theta: 225.0, Hx: 4.25, Hy:-0.5, Nosc: 28
LastIndex:  125
End State! Theta: 225.0, Hx: 4.25, Hy:0.0, Nosc: 28
LastIndex:  141
End State!

/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:85: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


### PLOT 2D PHASE DIAGRAMS: CONSTANT HX
#### For each Re

In [255]:
#Here are the sims which have been corrected from NOTA to the closest pair configuration
#These have been reviewed manually and the corrected state values will be included in a list
#We will then correct the values in our allData dataframe for the selected (parHx,parHy,Theta) sims
Re2correctparHx     = [ 1.25, 1.75, 1.75, 2.75, 3.25, 3.25, 3.25, 3.75, 3.75, 4.25,
                        5.25, 6.25, 6.25, 3.75, 2.75, 1.25, 0.25, 0.75, 0.25, 3.25,
                        5.25, 5.25, 4.25]
Re2correctparHy     = [ -3.5,  3.5, -4.5, -1.5,  1.5,  2.5, -2.5, -2.5,  1.5, -3.5,
                        -1.5,  1.5,  2.5, -0.5, -4.5, -3.5, -4.5, -4.5, -4.5, -2.5,
                         0.0,  0.0,  0.0]
Re2correctparTheta  = [112.5, 90.0,157.5,157.5,225.0,337.5,337.5,135.0,315.0,202.5,
                       202.5,337.5,337.5,337.5,112.5,112.5,112.5, 90.0, 90.0,202.5,
                         0.0,270.0,270.0]
Re2correctedState   = [    4,    0,    2,    2,    2,    1,    1,    4,    1,    2,
                           2,    1,    1,    1,    4,    4,    4,    4,    4,    2,
                           0,    4,    4]
Re2Dict = {'parHx':Re2correctparHx,'parHy':Re2correctparHy,'parThetaBW':Re2correctparTheta,'State':Re2correctedState}
correctRe2Data = pd.DataFrame(data=Re2Dict)
correctRe2Data['color'] = 's'
correctRe2Data['color_r'] = 0
correctRe2Data['color_g'] = 0
correctRe2Data['color_b'] = 0
#Add colors to database
for idx in range(len(correctRe2Data['State'])):
    State = correctRe2Data.loc[idx,'State']
    if State == 0:
        correctRe2Data.loc[idx,'color'] = 'gray'
        color_value = matplotlib.colors.to_rgb('gray')
        correctRe2Data.loc[idx,'color_r'] = color_value[0]
        correctRe2Data.loc[idx,'color_g'] = color_value[1]
        correctRe2Data.loc[idx,'color_b'] = color_value[2]
    elif State == 1:
        correctRe2Data.loc[idx,'color'] = 'blue'
        correctRe2Data.loc[idx,'color_r'] = 117
        correctRe2Data.loc[idx,'color_g'] = 112
        correctRe2Data.loc[idx,'color_b'] = 179
    elif State == 2:
        correctRe2Data.loc[idx,'color'] = 'green'
        correctRe2Data.loc[idx,'color_r'] = 27
        correctRe2Data.loc[idx,'color_g'] = 158
        correctRe2Data.loc[idx,'color_b'] = 119
    elif State == 3:
        correctRe2Data.loc[idx,'color'] = 'orange'
        correctRe2Data.loc[idx,'color_r'] = 217
        correctRe2Data.loc[idx,'color_g'] = 95
        correctRe2Data.loc[idx,'color_b'] = 2
    elif State == 4:
        correctRe2Data.loc[idx,'color'] = 'yellow'
        correctRe2Data.loc[idx,'color_r'] = 230
        correctRe2Data.loc[idx,'color_g'] = 171
        correctRe2Data.loc[idx,'color_b'] = 2

Re7correctparHx     = [ 0.25, 0.75, 1.25, 1.75, 2.25, 2.25, 2.75, 2.75, 2.75, 2.75,
                        3.25, 3.25, 3.25, 3.75, 3.75, 3.75, 3.75, 3.75, 4.25, 4.25,
                        4.25, 4.25, 4.25, 4.25, 4.25, 4.25, 4.25, 4.25, 4.25, 4.75,
                        4.75, 4.75, 4.75, 4.75, 4.75, 4.75, 4.75, 4.75, 4.75, 4.75,
                        4.75, 4.75, 4.75, 5.25, 5.25, 5.25, 5.25, 5.25, 5.25, 5.25,
                        5.25, 5.25, 5.25, 5.25, 5.25, 5.75, 5.75, 5.75, 5.75, 5.75,
                        5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 6.25, 6.25, 6.25, 6.25,
                        6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25,
                        6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 0.25, 0.25]
Re7correctparHy     = [  4.5,  4.5,  4.5,  4.5,  4.5,  4.5, -4.5,  4.5,  4.5,  2.5,
                         4.5,  4.5,  2.5,  4.5,  4.5, -1.5,  0.5,  4.5,  4.5, -4.5,
                         4.5, -3.5,  4.5,  3.5, -0.5,  2.5,  1.5, -4.5,  3.5,  4.5,
                        -4.5,  4.5, -1.5, -3.5,  4.5,  3.5,  2.5,  2.5, -0.5,  1.5,
                         0.5,  1.5,  3.5,  4.5, -4.5,  4.5,  3.5, -0.5, -3.5,  1.5,
                        -2.5,  0.5,  1.5,  3.5,  3.5,  4.5, -4.5,  3.5,  2.5, -0.5,
                        -2.5, -3.5, -4.5,  1.5, -4.5,  3.5,  4.5, -3.5, -4.5,  3.5,
                         2.5, -0.5, -2.5, -3.5, -4.5, -3.5,  0.5,  1.5,  0.5, -2.5,
                        -3.5, -4.5,  3.5,  2.5,  4.5,  3.5,  4.5,  3.5]
Re7correctparTheta  = [  0.0, 67.5, 67.5, 67.5, 22.5, 67.5, 45.0, 90.0,157.5,202.5,
                        45.0,112.5,180.0, 45.0, 67.5, 67.5,135.0,225.0,  0.0,  0.0,
                        22.5, 22.5, 45.0, 90.0, 90.0,135.0,135.0,157.5,202.5,  0.0,
                         0.0, 22.5, 22.5, 22.5, 45.0, 45.0, 45.0, 67.5, 67.5, 90.0,
                        90.0,157.5,202.5,  0.0,  0.0, 22.5, 22.5, 22.5, 22.5, 45.0,
                        67.5,135.0,157.5,202.5,337.5,  0.0,  0.0, 22.5, 22.5, 22.5,
                        22.5, 22.5, 22.5,180.0,180.0,202.5,  0.0,  0.0,  0.0, 22.5,
                        22.5, 22.5, 22.5, 22.5, 22.5, 45.0,157.5,180.0,180.0,180.0,
                       180.0,180.0,202.5,202.5,225.0,337.5,270.0,270.0]
Re7correctedState   = [    2,    0,    0,    1,    1,    0,    2,    0,    0,    4,
                           1,    0,    0,    1,    1,    0,    4,    4,    1,    1,
                           1,    1,    1,    3,    0,    0,    0,    4,    4,    1,
                           1,    1,    0,    1,    1,    1,    0,    0,    0,    3,
                           3,    4,    4,    1,    1,    1,    1,    1,    1,    0,
                           4,    4,    4,    4,    1,    1,    1,    1,    1,    0,
                           1,    1,    2,    4,    4,    4,    1,    1,    1,    1,
                           1,    0,    1,    1,    1,    1,    4,    4,    4,    4,
                           4,    4,    4,    4,    4,    1,    0,    3]
Re7Dict = {'parHx':Re7correctparHx,'parHy':Re7correctparHy,'parThetaBW':Re7correctparTheta,'State':Re7correctedState}
correctRe7Data = pd.DataFrame(data=Re7Dict) 
correctRe7Data['color'] = 's'
correctRe7Data['color_r'] = 0
correctRe7Data['color_g'] = 0
correctRe7Data['color_b'] = 0
#Add colors to database
for idx in range(len(correctRe7Data['State'])):
    State = correctRe7Data.loc[idx,'State']
    if State == 0:
        correctRe7Data.loc[idx,'color'] = 'gray'
        color_value = matplotlib.colors.to_rgb('gray')
        correctRe7Data.loc[idx,'color_r'] = color_value[0]
        correctRe7Data.loc[idx,'color_g'] = color_value[1]
        correctRe7Data.loc[idx,'color_b'] = color_value[2]
    elif State == 1:
        correctRe7Data.loc[idx,'color'] = 'blue'
        correctRe7Data.loc[idx,'color_r'] = 117
        correctRe7Data.loc[idx,'color_g'] = 112
        correctRe7Data.loc[idx,'color_b'] = 179
    elif State == 2:
        correctRe7Data.loc[idx,'color'] = 'green'
        correctRe7Data.loc[idx,'color_r'] = 27
        correctRe7Data.loc[idx,'color_g'] = 158
        correctRe7Data.loc[idx,'color_b'] = 119
    elif State == 3:
        correctRe7Data.loc[idx,'color'] = 'orange'
        correctRe7Data.loc[idx,'color_r'] = 217
        correctRe7Data.loc[idx,'color_g'] = 95
        correctRe7Data.loc[idx,'color_b'] = 2
    elif State == 4:
        correctRe7Data.loc[idx,'color'] = 'pink'
        correctRe7Data.loc[idx,'color_r'] = 231
        correctRe7Data.loc[idx,'color_g'] = 41
        correctRe7Data.loc[idx,'color_b'] = 138
        
#state = 4 = SF
Re10correctparHx    = [ 0.25, 0.25, 0.25, 0.75, 1.25, 2.25, 2.25, 2.75, 3.25, 3.25,
                        3.25, 3.75, 3.75, 4.25, 4.25, 4.75, 4.75, 4.75, 4.75, 4.75,
                        4.75, 5.25, 5.25, 5.25, 5.25, 5.25, 5.75, 5.75, 5.75, 5.75,
                        5.75, 5.75, 5.75, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25, 6.25,
                        6.25, 6.25, 6.25, 6.25, 6.25, 3.75]
Re10correctparHy    = [  4.5, -4.5,  4.5,  4.5,  4.5,  4.5,  4.5,  4.5, -2.5,  4.5,
                         3.5,  1.5,  2.5,  4.5,  3.5,  4.5,  1.5,  4.5,  3.5,  2.5,
                         3.5, -2.5,  4.5,  4.5,  0.5,  3.5, -2.5,  4.5,  3.5, -0.5,
                         3.5,  3.5,  2.5,  3.5, -2.5, -3.5,  3.5,  2.5,  0.5, -1.5,
                         2.5,  0.5, -0.5,  3.5,  1.5,  0.0]
Re10correctparTheta = [  0.0,  0.0,270.0,247.5,180.0, 45.0,225.0,225.0, 45.0,202.5,
                       202.5,157.5,180.0,180.0,180.0, 90.0,112.5,157.5,157.5,157.5,
                       337.5, 45.0, 67.5, 90.0,112.5,157.5, 45.0, 67.5, 90.0, 90.0,
                       112.5,135.0,135.0, 45.0, 45.0, 45.0, 67.5, 67.5, 67.5, 67.5,
                        90.0, 90.0, 90.0,112.5,112.5,292.5]
Re10correctedState  = [    2,    2,    3,    3,    3,    1,    3,    0,    1,    0,
                           3,    3,    0,    0,    0,    3,    3,    3,    3,    3,
                           4,    1,    0,    0,    1,    0,    1,    0,    1,    0,
                           0,    0,    3,    0,    0,    0,    1,    1,    3,    0,
                           0,    1,    0,    0,    0,    1]
Re10Dict = {'parHx':Re10correctparHx,'parHy':Re10correctparHy,'parThetaBW':Re10correctparTheta,'State':Re10correctedState}
correctRe10Data = pd.DataFrame(data=Re10Dict)
correctRe10Data['color'] = 's'
correctRe10Data['color_r'] = 0
correctRe10Data['color_g'] = 0
correctRe10Data['color_b'] = 0
#Add colors to database
for idx in range(len(correctRe10Data['State'])):
    State = correctRe10Data.loc[idx,'State']
    if State == 0:
        correctRe10Data.loc[idx,'color'] = 'gray'
        color_value = matplotlib.colors.to_rgb('gray')
        correctRe10Data.loc[idx,'color_r'] = color_value[0]
        correctRe10Data.loc[idx,'color_g'] = color_value[1]
        correctRe10Data.loc[idx,'color_b'] = color_value[2]
    elif State == 1:
        correctRe10Data.loc[idx,'color'] = 'blue'
        correctRe10Data.loc[idx,'color_r'] = 117
        correctRe10Data.loc[idx,'color_g'] = 112
        correctRe10Data.loc[idx,'color_b'] = 179
    elif State == 2:
        correctRe10Data.loc[idx,'color'] = 'green'
        correctRe10Data.loc[idx,'color_r'] = 27
        correctRe10Data.loc[idx,'color_g'] = 158
        correctRe10Data.loc[idx,'color_b'] = 119
    elif State == 3:
        correctRe10Data.loc[idx,'color'] = 'orange'
        correctRe10Data.loc[idx,'color_r'] = 217
        correctRe10Data.loc[idx,'color_g'] = 95
        correctRe10Data.loc[idx,'color_b'] = 2
    elif State == 4:
        correctRe10Data.loc[idx,'color'] = 'pink'
        correctRe10Data.loc[idx,'color_r'] = 231
        correctRe10Data.loc[idx,'color_g'] = 41
        correctRe10Data.loc[idx,'color_b'] = 138
        
Re25correctparHx    = [ 4.25, 4.25, 4.25]
Re25correctparHy    = [ -4.5,  3.5,  3.5]
Re25correctparTheta = [ 67.5,157.5,337.5]
Re25correctedState  = [    0,    3,    4]
Re25Dict = {'parHx':Re25correctparHx,'parHy':Re25correctparHy,'parThetaBW':Re25correctparTheta,'State':Re25correctedState}
correctRe25Data = pd.DataFrame(data=Re25Dict)
correctRe25Data['color'] = 's'
correctRe25Data['color_r'] = 0
correctRe25Data['color_g'] = 0
correctRe25Data['color_b'] = 0
#Add colors to database
for idx in range(len(correctRe25Data['State'])):
    State = correctRe25Data.loc[idx,'State']
    if State == 0:
        correctRe25Data.loc[idx,'color'] = 'gray'
        color_value = matplotlib.colors.to_rgb('gray')
        correctRe25Data.loc[idx,'color_r'] = color_value[0]
        correctRe25Data.loc[idx,'color_g'] = color_value[1]
        correctRe25Data.loc[idx,'color_b'] = color_value[2]
    elif State == 1:
        correctRe25Data.loc[idx,'color'] = 'blue'
        correctRe25Data.loc[idx,'color_r'] = 117
        correctRe25Data.loc[idx,'color_g'] = 112
        correctRe25Data.loc[idx,'color_b'] = 179
    elif State == 3:
        correctRe25Data.loc[idx,'color'] = 'orange'
        correctRe25Data.loc[idx,'color_r'] = 217
        correctRe25Data.loc[idx,'color_g'] = 95
        correctRe25Data.loc[idx,'color_b'] = 2
    elif State == 4:
        correctRe25Data.loc[idx,'color'] = 'pink'
        correctRe25Data.loc[idx,'color_r'] = 231
        correctRe25Data.loc[idx,'color_g'] = 41
        correctRe25Data.loc[idx,'color_b'] = 138
      
    

In [258]:
%matplotlib inline
from matplotlib.patches import Circle
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap

def AddDiscsToPlot(ax,pos):
    global RADIUSLARGE,RADIUSSMALL
    gridLarge = 1.0
    gridSmall = RADIUSSMALL/RADIUSLARGE
    #Add Discs
    circle1 = Circle((pos.loc[0,'aXU'], pos.loc[0,'aYU']), gridLarge, facecolor=(0.0,)*3,
                     linewidth=1,alpha=1.0,zorder=6)
    ax.add_patch(circle1)
    circle2 = Circle((pos.loc[0,'aXL'], pos.loc[0,'aYL']), gridSmall, facecolor=(0.0,)*3,
                     linewidth=1,alpha=1.0,zorder=6)
    ax.add_patch(circle2)
    #Add Swimmer "springs"
    ax.plot([pos.loc[0,'aXU'],pos.loc[0,'aXL']],
            [pos.loc[0,'aYU'],pos.loc[0,'aYL']],
            color='black',linewidth=1,zorder=5)
    #Swimmer 2
    circle3 = Circle((pos.loc[0,'bXU'], pos.loc[0,'bYU']), gridLarge, facecolor=(0.5,)*3,
                             linewidth=1,alpha=1.0,zorder=6)
    ax.add_patch(circle3)
    circle4 = Circle((pos.loc[0,'bXL'], pos.loc[0,'bYL']), gridSmall, facecolor=(0.5,)*3,
                     linewidth=1,alpha=1.0,zorder=6)
    ax.add_patch(circle4)
    #Add swimmer "springs"
    ax.plot([pos.loc[0,'bXU'],pos.loc[0,'bXL']],
            [pos.loc[0,'bYU'],pos.loc[0,'bYL']],
            color=(0.5,)*3,linewidth=1,zorder=6)
    return

def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
    return ax

def CalcSphereCenters(globalShift,number,angle,Hy,Hx):
    #global Hx
    if number == 1:
        U = globalShift + np.array([-0.5*Hx,0.5-0.5*Hy])
        L = globalShift + np.array([-0.5*Hx,-2.0-0.5*Hy])
        return U,L
    elif number == 2:
        U = np.array([0.0,0.5])
        L = np.array([0.0,-2.0])
        rotU = Rotate(U,angle*np.pi/180.0)
        rotL = Rotate(L,angle*np.pi/180.0)
        rotU = rotU + globalShift + np.array([0.5*Hx,0.5*Hy])
        rotL = rotL + globalShift + np.array([0.5*Hx,0.5*Hy])
        return rotU,rotL

def Construct_Image_Data(data,nrows,ncols):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    #xval = np.linspace(0.0,360.0,nrows)
    xval = np.linspace(0.0,337.5,nrows)
    yval = np.linspace(-4.5,4.5,ncols-1)
    yval = np.append(yval,0.0)
    yval = np.sort(yval)
    #print(xval)
    #print(yval)
    
    image = np.ones((ncols,nrows,3)) #idx, jdx, r, g, b
    
    for idx in range(len(xval)):
        xData = data[data['parThetaBW'] <= xval[idx] + 0.01].copy()
        xData = xData[xData['parThetaBW'] >= xval[idx] - 0.01].copy()
        for jdx in range(len(yval)):
            df = xData[xData['parHy'] <= yval[jdx]+0.01].copy()
            df = df[df['parHy'] >= yval[jdx]-0.01].copy()
            #print(len(df['parHy']))
            if(len(df['parHy']) == 1):
                df = df.reset_index(drop=True)
                if df.loc[0,'State'] == 0:
                    #Diverging
                    image[jdx,idx,:] = [df['color_r'].values,df['color_g'].values,df['color_b'].values]               
                else:
                    image[jdx,idx,:] = [df['color_r'].values/255.0,df['color_g'].values/255.0,df['color_b'].values/255.0]               
            else:
                image[jdx,idx,:] = (1.0,)*3 if (idx+jdx)%2 else (0.75,)*3    
    return image

def CheckForNOTA(ax,data,viableConfigs):
    testDict = {'Hx':data['parHx'],'Hy':data['parHy'],'Theta':data['parThetaBW']}
    testData = pd.DataFrame(data=testDict)
    testData = testData.sort_values(by=['Hy','Theta'])
    testData = testData.reset_index(drop=True)
    testData['comb'] = 's'
    for idx in range(len(testData['Hy'])):
        testData = testData.set_value(idx,'comb', (np.round(testData.loc[idx,'Hy'],2),np.round(testData.loc[idx,'Theta'],2)))
    #Do the same for viable configs
    viableConfigs['comb'] = 's'
    for idx in range(len(viableConfigs['Hy'])):
        viableConfigs = viableConfigs.set_value(idx,'comb', (np.round(viableConfigs.loc[idx,'Hy'],2),np.round(viableConfigs.loc[idx,'Theta'],2)))

    unique = viableConfigs[~viableConfigs['comb'].isin(testData['comb'])]
    #print(unique)
    unique = unique.reset_index(drop=True)
    if len(unique['comb']) > 0:
        for idx in range(len(unique['comb'])):
            ax.add_patch(patches.Rectangle(((boxWidth*unique.loc[idx,'Theta']/22.5-0.5*boxWidth), (boxHeight*(unique.loc[idx,'Hy']+4.5)-0.5*boxHeight)), boxWidth, boxHeight, 
                         hatch='xx', fill=False, snap=False, linewidth=0, color='k',zorder=7))
    return ax

def AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,alpha):
    Value = np.zeros((ncols,nrows))
    xval = np.linspace(-0.5*boxWidth,nrows*boxWidth-0.5*boxWidth,nrows+1)
    yval = np.linspace(-0.5*boxHeight,ncols*boxHeight-0.5*boxHeight,ncols+1)
    mx,my = np.meshgrid(xval,yval)
    colors = [(1,1,1,1), (0,0,0,0)]
    cm = LinearSegmentedColormap.from_list('custom', colors, N=256, gamma=0)
    ax.pcolor(mx,my,Value, edgecolors='k', linewidths=1,cmap=cm,alpha=alpha)
    return ax

#Here we are going to plot the pair of swimmers in a grid formation 
#for each Hy and Theta chosen from the list
#Trial and error baby!

#All InitConfigs
#gridHxList = [6.25,5.75,5.25,4.75,4.25,3.75,3.25,2.75,2.25,1.75,1.25,0.75,0.25]
gridHxList=[4.25]
gridHyList = [-4.5,-3.5,-2.5,-1.5,-0.5,0.0,0.5,1.5,2.5,3.5,4.5]
gridThetaList = [0.0,22.5,45.0,67.5,90.0,112.5,135.0,157.5,180.0,202.5,225.0,247.5,270.0,292.5,315.0,337.5]#,360.0]

allViableConfigs = pd.read_csv(cwd_PYTHON+'/allViableConfigs.csv',delimiter=' ')
#chosenThetaData = allViableConfigs[allViableConfigs['Theta'] <= 0.1].copy()
#chosenThetaData = chosenThetaData[chosenThetaData['Theta'] >= -0.1].copy()
#chosenThetaData['Theta'] = 360.0
#allViableConfigs = pd.concat([allViableConfigs,chosenThetaData],ignore_index=True)

posDict = {'aXU':[],'aYU':[],'aXL':[],'aYL':[],'bXU':[],'bYU':[],'bXL':[],'bYL':[]}
pos = pd.DataFrame(data=posDict)

boolSwimmers = 2
#ReList=['2','7','10']
ReList=['2','7','10','25']
for Re in ReList:
    print('Re = ',Re)
    allData = pd.read_csv(cwd_PYTHON+'/SimData/allEndData_Re{0}_.csv'.format(Re),delimiter=' ')
    endDict = {'parThetaBW':allData['parThetaBW'],'parHx':allData['parHx'],'parHy':allData['parHy'],
               'State':allData['State'],'color_tuple':allData['color_tuple'],
               'color_r':allData['color_r'],'color_g':allData['color_g'],'color_b':allData['color_b']}
    endData = pd.DataFrame(data=endDict)
    endData = endData.sort_values(by=['parHx','parThetaBW','parHy'])
    endData = endData.reset_index(drop=True)
    #Add Manual Dataframes based on Re
    if int(Re) == 2:
        endData = pd.concat([endData,correctRe2Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 10:
        endData = pd.concat([endData,correctRe10Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 7:
        endData = pd.concat([endData,correctRe7Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 25:
        endData = pd.concat([endData,correctRe25Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    '''
    #Change color of L-Shape to aqua
    if int(Re) == 2:
        endData['color_r'] = np.where(endData['State'] == 4, 102,endData['color_r'])
        endData['color_g'] = np.where(endData['State'] == 4, 166,endData['color_g'])
        endData['color_b'] = np.where(endData['State'] == 4, 130,endData['color_b'])
    '''
    #chosenThetaData = endData[endData['parThetaBW'] <= 0.1].copy()
    #chosenThetaData = chosenThetaData[chosenThetaData['parThetaBW'] >= -0.1].copy()
    #chosenThetaData['parThetaBW'] = 360.0
    #endData = pd.concat([endData,chosenThetaData],ignore_index=True)
    
    for Hx in gridHxList:
        viableConfigs = allViableConfigs[allViableConfigs['Hx'] == Hx].copy()
        viableConfigs = viableConfigs.reset_index(drop=True)
        
        data = endData[endData['parHx'] <= float(Hx)+0.01].copy()
        data = data[data['parHx'] >= float(Hx)-0.01].copy()
        data = data.sort_values(by=['parThetaBW','parHy'])
        data = data.reset_index(drop=True)
        
        fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(14,8),dpi=250)
        if boolSwimmers == 2:
            if int(Re) == 2:
                ax.set_title('(a) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 7:
                ax.set_title('(b) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 10:
                ax.set_title('(c) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 25:
                ax.set_title('(d) Re = {0}'.format(Re),fontsize=24,**csfont)
            #ax.set_ylabel(r'$\hat{H}_y$',fontsize=24,**csfont)
        else:
            ax.set_title(r'$\hat{H}_x$ = %.2f'%(Hx),fontsize=24,**csfont)
        ax.set_ylabel(r'$\hat{H}_y$',fontsize=24,**csfont)
        ax.set_xlabel(r'$\theta$',fontsize=24,**csfont)
        #Add Image Color from data
        nrows, ncols = len(gridThetaList), len(gridHyList)
        image = Construct_Image_Data(data,nrows,ncols)
        boxHeight, boxWidth = 11.0, 11.0
        ax.imshow(image,cmap=None,extent=(-0.5*boxWidth,(nrows-0.5)*boxWidth,-0.5*boxHeight,(ncols-0.5)*boxHeight),alpha=0.5,origin='lower',zorder=2)#,aspect=boxHeight/boxWidth)
        #1)Find pos of first swimmer. Will use as a global shift
        for idx in range(len(gridHyList)):
            Hy = gridHyList[idx]
            valHy = Hy
            configs = viableConfigs[viableConfigs['Hy'] == valHy].copy()
            for jdx in range(len(gridThetaList)):
                Theta = gridThetaList[jdx]
                check = configs[configs['Theta'] == Theta].copy()
                #print('configs = ',configs)
                if len(gridThetaList) > len(gridHyList):
                    #globalShift = np.array([jdx*factor*spacing,idx*factor])
                    globalShift = np.array([(boxWidth)*jdx,(boxHeight)*idx])
                else:
                    globalShift = np.array([boxWidth*jdx,boxHeight*idx])
                    #globalShift = np.array([jdx*factor,idx*factor/spacing])
                #Calculate pos of first swimmer
                aU, aL = CalcSphereCenters(globalShift,1,0.0,Hy,Hx)
                bU, bL = CalcSphereCenters(globalShift,2,Theta,Hy,Hx)
                pos.loc[0,'aXU'], pos.loc[0,'aYU'] = aU[0], aU[1]
                pos.loc[0,'aXL'], pos.loc[0,'aYL'] = aL[0], aL[1]
                pos.loc[0,'bXU'], pos.loc[0,'bYU'] = bU[0], bU[1]
                pos.loc[0,'bXL'], pos.loc[0,'bYL'] = bL[0], bL[1]
                #print(pos)
                #Plot onto grid
                if boolSwimmers:
                    AddDiscsToPlot(ax,pos)
                if check.empty:
                    #print('Hx = {0}: Hy = {1}: Theta = {2}'.format(Hx,Hy,Theta))
                    ax.add_patch(patches.Rectangle(((globalShift[0]-5.5), (globalShift[1]-5.5)), 11.0, 11.0, 
                             hatch='//', fill=False, snap=False, linewidth=0, color='r', alpha=0.25,zorder=7))

        #Check if there are any NOTA cases left
        #ax = CheckForNOTA(ax,data,viableConfigs)

        #Insert black borders
        ax = AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,0.25)
        
        ax.set_aspect('equal')
        ax.tick_params(which='major',axis='both',direction='in',length=1,width=1,zorder=1)
        #ax.xaxis.set_major_locator(MultipleLocator(11.0))
        #ax.yaxis.set_major_locator(MultipleLocator(11.0))
        ax.xaxis.set_major_locator(MultipleLocator(22.0))
        ax.yaxis.set_major_locator(MultipleLocator(11.0))
        ax.set_yticklabels(['temp']+gridHyList)#[::2])
        ax.set_xticklabels(['temp']+[0,45,90,135,180,225,270,315])
        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(18)
        #ax.axis([-5,(len(ThetaList)-1)*factor*spacing+5,-factor-10,(len(HyList)-2)*factor+10])
        ax = set_size(0.4*17,4,ax)
        #fig.tight_layout()
        #ax = set_size(0.6*17,6,ax)
        if boolSwimmers == 1:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hx/Re{0}/".format(Re)
        elif boolSwimmers == 0:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hx/Re{0}/NoSwim/".format(Re)
        elif boolSwimmers == 2:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hx/".format(Re)
        pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
        if boolSwimmers == 2:
            fig.savefig(strDir+'main_Re{0}_Constant_Hx{1}.png'.format(Re,Hx))
        else:
            fig.savefig(strDir+'Re{0}_Constant_Hx{1}_.png'.format(Re,Hx))
        #plt.show()
        fig.clf()
        plt.close
        
        print('Hx = {0} is complete!'.format(Hx))



Re =  2


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:157: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 4.25 is complete!
Re =  7


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:163: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 4.25 is complete!
Re =  10


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:160: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 4.25 is complete!
Re =  25


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:166: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 4.25 is complete!


<Figure size 2193.55x1324.5 with 0 Axes>

### Plot Hx vs Theta for specific Hy

In [191]:
def Construct_Hy_Image_Data(data,nrows,ncols):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    #xval = np.linspace(0.0,360.0,nrows)
    xval = np.linspace(0.0,337.5,nrows)
    #yval = np.linspace(0.25,6.25,ncols)
    yval = np.linspace(2.25,6.25,ncols)
    #yval = np.append(yval,0.0)
    #yval = np.sort(yval)
    #print(xval)
    #print(yval)
    
    image = np.ones((ncols,nrows,3)) #idx, jdx, r, g, b
    
    for idx in range(len(xval)):
        xData = data[data['parThetaBW'] <= xval[idx] + 0.01].copy()
        xData = xData[xData['parThetaBW'] >= xval[idx] - 0.01].copy()
        for jdx in range(len(yval)):
            df = xData[xData['parHx'] <= yval[jdx]+0.01].copy()
            df = df[df['parHx'] >= yval[jdx]-0.01].copy()
            #print(len(df['parHy']))
            if(len(df['parHx']) == 1):
                df = df.reset_index(drop=True)
                if df.loc[0,'State'] == 0:
                    #Diverging
                    image[jdx,idx,:] = [df['color_r'].values,df['color_g'].values,df['color_b'].values]               
                else:
                    image[jdx,idx,:] = [df['color_r'].values/255.0,df['color_g'].values/255.0,df['color_b'].values/255.0]               
            else:
                image[jdx,idx,:] = (1.0,)*3 if (idx+jdx)%2 else (0.75,)*3    
    return image

#All InitConfigs
#gridHxList = [0.25,0.75,1.25,1.75,2.25,2.75,3.25,3.75,4.25,4.75,5.25,5.75,6.25]
gridHxList = [2.25,2.75,3.25,3.75,4.25,4.75,5.25,5.75,6.25]
#gridHyList = [-4.5,-3.5,-2.5,-1.5,-0.5,0.0,0.5,1.5,2.5,3.5,4.5]
gridHyList = [0.0]
gridThetaList = [0.0,22.5,45.0,67.5,90.0,112.5,135.0,157.5,180.0,202.5,225.0,247.5,270.0,292.5,315.0,337.5]#,360.0]

allViableConfigs = pd.read_csv(cwd_PYTHON+'/allViableConfigs.csv',delimiter=' ')
#chosenThetaData = allViableConfigs[allViableConfigs['Theta'] <= 0.1].copy()
#chosenThetaData = chosenThetaData[chosenThetaData['Theta'] >= -0.1].copy()
#chosenThetaData['Theta'] = 360.0
#allViableConfigs = pd.concat([allViableConfigs,chosenThetaData],ignore_index=True)

posDict = {'aXU':[],'aYU':[],'aXL':[],'aYL':[],'bXU':[],'bYU':[],'bXL':[],'bYL':[]}
pos = pd.DataFrame(data=posDict)

boolSwimmers = 2
ReList=['2','7','10']
#ReList=['2','7','10','25']
for Re in ReList:
    print('Re = ',Re)
    allData = pd.read_csv(cwd_PYTHON+'/SimData/allEndData_Re{0}_.csv'.format(Re),delimiter=' ')
    endDict = {'parThetaBW':allData['parThetaBW'],'parHx':allData['parHx'],'parHy':allData['parHy'],
               'State':allData['State'],'color_tuple':allData['color_tuple'],
               'color_r':allData['color_r'],'color_g':allData['color_g'],'color_b':allData['color_b']}
    endData = pd.DataFrame(data=endDict)
    endData = endData.sort_values(by=['parHx','parThetaBW','parHy'])
    endData = endData.reset_index(drop=True)
    #Add Manual Dataframes based on Re
    if int(Re) == 2:
        endData = pd.concat([endData,correctRe2Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 10:
        endData = pd.concat([endData,correctRe10Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 7:
        endData = pd.concat([endData,correctRe7Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 25:
        endData = pd.concat([endData,correctRe25Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    '''
    #Change color of L-Shape to aqua
    if int(Re) == 2:
        endData['color_r'] = np.where(endData['State'] == 4, 102,endData['color_r'])
        endData['color_g'] = np.where(endData['State'] == 4, 166,endData['color_g'])
        endData['color_b'] = np.where(endData['State'] == 4, 130,endData['color_b'])
    '''
    #chosenThetaData = endData[endData['parThetaBW'] <= 0.1].copy()
    #chosenThetaData = chosenThetaData[chosenThetaData['parThetaBW'] >= -0.1].copy()
    #chosenThetaData['parThetaBW'] = 360.0
    #endData = pd.concat([endData,chosenThetaData],ignore_index=True)
    
    for Hy in gridHyList:
        viableConfigs = allViableConfigs[allViableConfigs['Hy'] == Hy].copy()
        viableConfigs = viableConfigs.reset_index(drop=True)
        
        data = endData[endData['parHy'] <= float(Hy)+0.01].copy()
        data = data[data['parHy'] >= float(Hy)-0.01].copy()
        data = data.sort_values(by=['parThetaBW','parHy'])
        data = data.reset_index(drop=True)
        
        fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(14,8),dpi=250)
        if boolSwimmers == 2:
            if int(Re) == 2:
                ax.set_title('(a) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 7:
                ax.set_title('(b) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 10:
                ax.set_title('(c) Re = {0}'.format(Re),fontsize=24,**csfont)
            elif int(Re) == 25:
                ax.set_title('(d) Re = {0}'.format(Re),fontsize=24,**csfont)
            #ax.set_ylabel(r'$\hat{H}_y$',fontsize=24,**csfont)
        else:
            ax.set_title(r'$\hat{H}_y$ = %.2f'%(Hy),fontsize=24,**csfont)
        ax.set_ylabel(r'$\hat{H}_x$',fontsize=24,**csfont)
        ax.set_xlabel(r'$\theta$',fontsize=24,**csfont)
        #Add Image Color from data
        nrows, ncols = len(gridThetaList), len(gridHxList)
        image = Construct_Hy_Image_Data(data,nrows,ncols)
        boxHeight, boxWidth = 11.0, 11.0
        ax.imshow(image,cmap=None,extent=(-0.5*boxWidth,(nrows-0.5)*boxWidth,-0.5*boxHeight,(ncols-0.5)*boxHeight),alpha=0.5,origin='lower',zorder=2)#,aspect=boxHeight/boxWidth)
        #1)Find pos of first swimmer. Will use as a global shift
        for idx in range(len(gridHxList)):
            Hx = gridHxList[idx]
            valHx = Hx
            configs = viableConfigs[viableConfigs['Hx'] == valHx].copy()
            for jdx in range(len(gridThetaList)):
                Theta = gridThetaList[jdx]
                check = configs[configs['Theta'] == Theta].copy()
                #print('configs = ',configs)
                if len(gridThetaList) > len(gridHyList):
                    #globalShift = np.array([jdx*factor*spacing,idx*factor])
                    globalShift = np.array([(boxWidth)*jdx,(boxHeight)*idx])
                else:
                    globalShift = np.array([boxWidth*jdx,boxHeight*idx])
                    #globalShift = np.array([jdx*factor,idx*factor/spacing])
                #Calculate pos of first swimmer
                aU, aL = CalcSphereCenters(globalShift,1,0.0,Hy,Hx)
                bU, bL = CalcSphereCenters(globalShift,2,Theta,Hy,Hx)
                pos.loc[0,'aXU'], pos.loc[0,'aYU'] = aU[0], aU[1]
                pos.loc[0,'aXL'], pos.loc[0,'aYL'] = aL[0], aL[1]
                pos.loc[0,'bXU'], pos.loc[0,'bYU'] = bU[0], bU[1]
                pos.loc[0,'bXL'], pos.loc[0,'bYL'] = bL[0], bL[1]
                #print(pos)
                #Plot onto grid
                if boolSwimmers:
                    AddDiscsToPlot(ax,pos)
                if check.empty:
                    #print('Hx = {0}: Hy = {1}: Theta = {2}'.format(Hx,Hy,Theta))
                    ax.add_patch(patches.Rectangle(((globalShift[0]-5.5), (globalShift[1]-5.5)), 11.0, 11.0, 
                             hatch='//', fill=False, snap=False, linewidth=0, color='r', alpha=0.25,zorder=7))

        #Check if there are any NOTA cases left
        #ax = CheckForNOTA(ax,data,viableConfigs)

        #Insert black borders
        ax = AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight)
        
        ax.set_aspect('equal')
        ax.tick_params(which='major',axis='both',direction='in',length=1,width=1,zorder=1)
        #ax.xaxis.set_major_locator(MultipleLocator(11.0))
        #ax.yaxis.set_major_locator(MultipleLocator(11.0))
        ax.xaxis.set_major_locator(MultipleLocator(22.0))
        ax.yaxis.set_major_locator(MultipleLocator(22.0))
        ax.set_yticklabels(['temp']+gridHxList[::2])
        ax.set_xticklabels(['temp']+[0,45,90,135,180,225,270,315])
        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(18)
        #ax.axis([-5,(len(ThetaList)-1)*factor*spacing+5,-factor-10,(len(HyList)-2)*factor+10])
        ax = set_size(0.4*17,4,ax)
        #fig.tight_layout()
        #ax = set_size(0.6*17,6,ax)
        if boolSwimmers == 1:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hy/Re{0}/".format(Re)
        elif boolSwimmers == 0:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hy/Re{0}/NoSwim/".format(Re)
        elif boolSwimmers == 2:
            strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/Constant_Hy/".format(Re)
        pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
        if boolSwimmers == 2:
            fig.savefig(strDir+'main_Re{0}_Constant_Hy{1}.png'.format(Re,Hy))
        else:
            fig.savefig(strDir+'Re{0}_Constant_Hy{1}_.png'.format(Re,Hy))
        #plt.show()
        fig.clf()
        plt.close
        
        print('Hy = {0} is complete!'.format(Hy))



Re =  2


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:63: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hy = 0.0 is complete!
Re =  7


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hy = 0.0 is complete!
Re =  10


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:66: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hy = 0.0 is complete!


<Figure size 2193.55x1324.5 with 0 Axes>

### Plot Hy vs Hx for each Theta. Theta will be indicated by a set up 16 squares inside the Hy,Hx square

In [228]:
def Construct_HyHx_Image_Data(data,nrows,ncols):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    #xval = np.linspace(0.0,360.0,nrows)
    val = np.linspace(0.0,337.5,nrows*ncols).reshape((4,4))
    image = np.ones((ncols,nrows,3)) #idx, jdx, r, g, b
    
    for idx in range(nrows):
        for jdx in range(ncols):
            theta = val[jdx,idx]
            df = data[data['parThetaBW'] <= theta + 0.01].copy()
            df = df[df['parThetaBW'] >= theta - 0.01].copy()
            if(len(df['parThetaBW']) == 1):
                df = df.reset_index(drop=True)
                if df.loc[0,'State'] == 0:
                    #Diverging
                    image[jdx,idx,:] = [df['color_r'].values,df['color_g'].values,df['color_b'].values]               
                else:
                    image[jdx,idx,:] = [df['color_r'].values/255.0,df['color_g'].values/255.0,df['color_b'].values/255.0]               
            else:
                image[jdx,idx,:] = (1.0,)*3 if (idx+jdx)%2 else (0.75,)*3
    return image

def AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,alpha):
    Value = np.zeros((ncols,nrows))
    xval = np.linspace(-0.5*boxWidth,nrows*boxWidth-0.5*boxWidth,nrows+1)
    yval = np.linspace(-0.5*boxHeight,ncols*boxHeight-0.5*boxHeight,ncols+1)
    mx,my = np.meshgrid(xval,yval)
    colors = [(1,1,1,1), (0,0,0,0)]
    cm = LinearSegmentedColormap.from_list('custom', colors, N=256, gamma=0)
    ax.pcolor(mx,my,Value, edgecolors='k', linewidths=1,cmap=cm,alpha=alpha)
    return ax

def AddInsideBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,alpha):
    Value = np.zeros((4*ncols,4*nrows))
    xval = np.linspace(-0.5*boxWidth,nrows*boxWidth-0.5*boxWidth,4*nrows+1)
    yval = np.linspace(-0.5*boxHeight,ncols*boxHeight-0.5*boxHeight,4*ncols+1)
    mx,my = np.meshgrid(xval,yval)
    colors = [(1,1,1,1), (0,0,0,0)]
    cm = LinearSegmentedColormap.from_list('custom', colors, N=256, gamma=0)
    ax.pcolor(mx,my,Value, edgecolors='k', linewidths=1,cmap=cm,alpha=alpha)
    return ax

#All InitConfigs
gridHxList = [0.25,0.75,1.25,1.75,2.25,2.75,3.25,3.75,4.25,4.75,5.25,5.75,6.25]
gridHyList = [-4.5,-3.5,-2.5,-1.5,-0.5,0.0,0.5,1.5,2.5,3.5,4.5]

allViableConfigs = pd.read_csv(cwd_PYTHON+'/allViableConfigs.csv',delimiter=' ')

posDict = {'aXU':[],'aYU':[],'aXL':[],'aYL':[],'bXU':[],'bYU':[],'bXL':[],'bYL':[]}
pos = pd.DataFrame(data=posDict)

boolSwimmers = 0
ReList=['2','7','10']
#ReList=['2','7','10','25']
for Re in ReList:
    print('Re = ',Re)
    allData = pd.read_csv(cwd_PYTHON+'/SimData/allEndData_Re{0}_.csv'.format(Re),delimiter=' ')
    endDict = {'parThetaBW':allData['parThetaBW'],'parHx':allData['parHx'],'parHy':allData['parHy'],
               'State':allData['State'],'color_tuple':allData['color_tuple'],
               'color_r':allData['color_r'],'color_g':allData['color_g'],'color_b':allData['color_b']}
    endData = pd.DataFrame(data=endDict)
    endData = endData.sort_values(by=['parHx','parThetaBW','parHy'])
    endData = endData.reset_index(drop=True)
    #Add Manual Dataframes based on Re
    if int(Re) == 2:
        endData = pd.concat([endData,correctRe2Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 10:
        endData = pd.concat([endData,correctRe10Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 7:
        endData = pd.concat([endData,correctRe7Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
    elif int(Re) == 25:
        endData = pd.concat([endData,correctRe25Data],ignore_index=True)
        endData = endData.drop_duplicates(subset=['parHx','parHy','parThetaBW'], keep="last")
        
    fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(8*1.25,8),dpi=250)
    ax.set_title(r'Re = {0}'.format(Re),fontsize=24,**csfont)
    ax.set_ylabel(r'$\hat{H}_y$',fontsize=24,**csfont)
    ax.set_xlabel(r'$\hat{H}_x$',fontsize=24,**csfont)
    xCount = 0
    for Hx in gridHxList:
        viableConfigs = allViableConfigs[allViableConfigs['Hx'] == Hx].copy()
        viableConfigs = viableConfigs.reset_index(drop=True)
        
        HxData = endData[endData['parHx'] <= float(Hx)+0.01].copy()
        HxData = HxData[HxData['parHx'] >= float(Hx)-0.01].copy()
        HxData = HxData.sort_values(by=['parThetaBW','parHy'])
        HxData = HxData.reset_index(drop=True)
        
        #Add Image Color from data
        #nrows, ncols = len(gridHxList), len(gridHyList)
        #image = Construct_HyHx_Image_Data(data,nrows,ncols)
        #boxHeight, boxWidth = 11.0, 11.0
        #ax.imshow(image,cmap=None,extent=(-0.5*boxWidth,(nrows-0.5)*boxWidth,-0.5*boxHeight,(ncols-0.5)*boxHeight),alpha=0.5,origin='lower',zorder=2)#,aspect=boxHeight/boxWidth)
        for idx in range(len(gridHyList)):
            Hy = gridHyList[idx]
            valHy = Hy
            data = HxData[HxData['parHy'] <= float(Hy)+0.01].copy()
            data = data[data['parHy'] >= float(Hy)-0.01].copy()
            data = data.sort_values(by=['parThetaBW'])
            data = data.reset_index(drop=True)
            configs = viableConfigs[viableConfigs['Hy'] == valHy].copy()
            #For each Hy,Hx get state for Theta
            #Plot State as image color
            globalShift = np.array([boxWidth*xCount,boxHeight*idx])
            #print('globalShift = ',globalShift)
            nrows, ncols = 4,4
            image = Construct_HyHx_Image_Data(data,nrows,ncols)
            boxHeight, boxWidth = 11.0, 11.0
            ax.imshow(image,cmap=None,
                      extent=(globalShift[0] -0.5*boxWidth,globalShift[0] + 0.5*boxWidth,
                              globalShift[1] -0.5*boxHeight,globalShift[1] + 0.5*boxHeight),
                      alpha=0.5,origin='upper',zorder=2)
            #plt.show()
            #Check if initial configuration overlaps
            
            for jdx in range(len(gridThetaList)):
                Theta = gridThetaList[jdx]
                check = configs[configs['Theta'] == Theta].copy()
                xidx = jdx % 4
                yidx = int(np.trunc(jdx/4))
                #print('xidx = {0}: yidx = {1}'.format(xidx,yidx))
                if check.empty:
                    #print('Hx = {0}: Hy = {1}: Theta = {2}'.format(Hx,Hy,Theta))
                    thetaWidth = boxWidth/4.0
                    thetaHeight = boxHeight/4.0
                    ax.add_patch(patches.Rectangle(((globalShift[0]-5.5 + float(xidx)*thetaWidth), (globalShift[1]-5.5 + float((3.0-yidx))*thetaHeight)), 
                                 thetaWidth, thetaHeight, hatch='////', fill=False, snap=False, 
                                 linewidth=0, color='r', alpha=0.25,zorder=7))
            
            #Check if there are any NOTA cases left
            #ax = CheckForNOTA(ax,data,viableConfigs)
            #print('Hy = {0} is complete!'.format(Hy))
        xCount += 1
        print('Hx = {0} is complete'.format(Hx))
    #Insert black borders
    nrows, ncols = len(gridHxList), len(gridHyList)
    boxHeight, boxWidth = 11.0, 11.0
    ax = AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,1.0)
    ax = AddInsideBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,0.25)
    ax.set_aspect('equal')
    ax.tick_params(which='major',axis='both',direction='in',length=1,width=1,zorder=1)
    #ax.xaxis.set_major_locator(MultipleLocator(11.0))
    ax.yaxis.set_major_locator(MultipleLocator(11.0))
    ax.xaxis.set_major_locator(MultipleLocator(22.0))
    #ax.yaxis.set_major_locator(MultipleLocator(22.0))
    ax.set_yticklabels(['temp']+gridHyList)
    ax.set_xticklabels(['temp']+gridHxList[::2])
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(18)
    #ax.axis([-5,(len(ThetaList)-1)*factor*spacing+5,-factor-10,(len(HyList)-2)*factor+10])
    #ax = set_size(4.0*13.0/11.0,4,ax)
    #fig.tight_layout()
    #ax = set_size(0.6*17,6,ax)
    strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/All/"
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    fig.savefig(strDir+'all_Re{0}_.png'.format(Re))
    #plt.show()
    fig.clf()
    plt.close




Re =  2


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:67: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 0.25 is complete
Hx = 0.75 is complete
Hx = 1.25 is complete
Hx = 1.75 is complete
Hx = 2.25 is complete
Hx = 2.75 is complete
Hx = 3.25 is complete
Hx = 3.75 is complete
Hx = 4.25 is complete
Hx = 4.75 is complete
Hx = 5.25 is complete
Hx = 5.75 is complete
Hx = 6.25 is complete
Re =  7


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:73: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 0.25 is complete
Hx = 0.75 is complete
Hx = 1.25 is complete
Hx = 1.75 is complete
Hx = 2.25 is complete
Hx = 2.75 is complete
Hx = 3.25 is complete
Hx = 3.75 is complete
Hx = 4.25 is complete
Hx = 4.75 is complete
Hx = 5.25 is complete
Hx = 5.75 is complete
Hx = 6.25 is complete
Re =  10


/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Hx = 0.25 is complete
Hx = 0.75 is complete
Hx = 1.25 is complete
Hx = 1.75 is complete
Hx = 2.25 is complete
Hx = 2.75 is complete
Hx = 3.25 is complete
Hx = 3.75 is complete
Hx = 4.25 is complete
Hx = 4.75 is complete
Hx = 5.25 is complete
Hx = 5.75 is complete
Hx = 6.25 is complete


<Figure size 2500x2000 with 0 Axes>

### Create Schematic for 1 Cell in above combined diagram

In [241]:
def Construct_Background_Image_Data(data,nrows,ncols):
    #From our color data and initial configurations, construct 2D mesh which has positions and colors for each position
    #If parHx and parHy do not exist, make color white
    image = np.ones((ncols,nrows,3)) #idx, jdx, r, g, b
    
    for idx in range(nrows):
        for jdx in range(ncols):
            image[jdx,idx,:] = (1.0,)*3 if (idx+jdx)%2 else (0.75,)*3
    return image

#All InitConfigs
valHx = 6.25
valHy = 0.0
gridThetaList = [0.0,22.5,45.0,67.5,90.0,112.5,135.0,157.5,180.0,202.5,225.0,247.5,270.0,292.5,315.0,337.5]#,360.0]

posDict = {'aXU':[],'aYU':[],'aXL':[],'aYL':[],'bXU':[],'bYU':[],'bXL':[],'bYL':[]}
pos = pd.DataFrame(data=posDict)

boolSwimmers = 1

fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(4,4),dpi=250)
#Add Image Color from data
nrows, ncols = 4, 4
image = Construct_Background_Image_Data(data,nrows,ncols)
boxHeight, boxWidth = 12.0, 12.0
ax.imshow(image,cmap=None,extent=(-0.5*boxWidth,(nrows-0.5)*boxWidth,-0.5*boxHeight,(ncols-0.5)*boxHeight),alpha=0.5,origin='lower',zorder=2)#,aspect=boxHeight/boxWidth)
for idx in range(len(gridThetaList)):
    Theta = gridThetaList[idx]
    xidx = idx % 4
    yidx = 3-int(np.trunc(idx/4))
    globalShift = np.array([(boxWidth)*xidx,(boxHeight)*yidx])
    aU, aL = CalcSphereCenters(globalShift,1,0.0,valHy,valHx)
    bU, bL = CalcSphereCenters(globalShift,2,Theta,valHy,valHx)
    pos.loc[0,'aXU'], pos.loc[0,'aYU'] = aU[0], aU[1]
    pos.loc[0,'aXL'], pos.loc[0,'aYL'] = aL[0], aL[1]
    pos.loc[0,'bXU'], pos.loc[0,'bYU'] = bU[0], bU[1]
    pos.loc[0,'bXL'], pos.loc[0,'bYL'] = bL[0], bL[1]
    AddDiscsToPlot(ax,pos)
#Insert black borders
ax = AddBlackBorders(ax,nrows,ncols,boxWidth,boxHeight,1.0)
ax.set_aspect('equal')
ax.tick_params(which='major',axis='both',direction='in',length=1,width=1,zorder=1)
#ax.xaxis.set_major_locator(MultipleLocator(11.0))
#ax.yaxis.set_major_locator(MultipleLocator(11.0))
#ax.xaxis.set_major_locator(MultipleLocator(22.0))
#ax.yaxis.set_major_locator(MultipleLocator(22.0))
ax.set_yticklabels(['temp'])
ax.set_xticklabels(['temp'])
for spine in ax.spines.values():
        spine.set_edgecolor('red')
#for label in (ax.get_xticklabels() + ax.get_yticklabels()):
#    label.set_fontsize(18)
#ax.axis([-5,(len(ThetaList)-1)*factor*spacing+5,-factor-10,(len(HyList)-2)*factor+10])
#ax = set_size(0.4*17,4,ax)
strDir = cwd_PYTHON+"/../Figures/PhaseSpace/2D_Diagrams/All/".format(Re)
fig.savefig(strDir+'InitConfigs_Hx_{0}_Hy_{1}_.png'.format(valHx,valHy))
fig.clf()
plt.close

<function matplotlib.pyplot.close(fig=None)>

<Figure size 1000x1000 with 0 Axes>

### Plot Ind Sim Trajectories

In [174]:
#Visualize Re=10, Theta = 0, Hx = 1.25, Hy = -5.5

def PlotSwimmers(data,trajData,Re,ax):
    global PERIOD,RADIUSLARGE,RADIUSSMALL
    #Data Values
    val1    = data['Hx']
    val2    = data['Hy']
    xU      = data['bXU']
    xL      = data['bXL']
    yU      = data['bYU']
    yL      = data['bYL']
    
    #GENERATE FIGURE
    #Phase Space  2D (Hy vs Hx)
    ax.plot(trajData['Hx'],trajData['Hy'],color='b',zorder=5)
    ax.plot(trajData['bXL'],trajData['bYL'],color='k',zorder=5)
    ax.scatter(data.loc[0,'Hx'],data.loc[0,'Hy'],c='tab:green',s=12,marker='s',zorder=6)
    ax.scatter(data.loc[0,'parHx'],data.loc[0,'parHy'],c='tab:red',s=12,marker='s',zorder=6)
    
    #Add Swimmer locations
    Circle1 = plt.Circle((0.0,0.5),1.0,color='k', clip_on=True)
    ax.add_artist(Circle1)
    Circle2 = plt.Circle((0.0,-2.0),0.5,color='k', clip_on=True)
    ax.add_artist(Circle2)
    Circle3 = plt.Circle((xU,yU),1.0,color=(0.5,)*3, clip_on=True)
    ax.add_artist(Circle3)
    Circle4 = plt.Circle((xL,yL),0.5,color=(0.5,)*3, clip_on=True)
    ax.add_artist(Circle4)
    
    #Add swimmers but now they are fully expanded
    AMP = 0.8
    xUA, yUA = xU + 0.2*AMP*data['nTx'], yU + 0.2*AMP*data['nTy']
    xLA, yLA = xL - 0.8*AMP*data['nTx'], yL - 0.8*AMP*data['nTy']
    Circle9 = plt.Circle((0.0,0.5+0.2*AMP),1.0, clip_on=True,fill=False,edgecolor='green')
    ax.add_artist(Circle9)
    Circle10 = plt.Circle((0.0,-2.0-0.8*AMP),0.5, clip_on=True,fill=False,edgecolor='green')
    ax.add_artist(Circle10)
    Circle11 = plt.Circle((xUA,yUA),1.0, clip_on=True,fill=False,edgecolor='red')
    ax.add_artist(Circle11)
    Circle12 = plt.Circle((xLA,yLA),0.5, clip_on=True,fill=False,edgecolor='red')
    ax.add_artist(Circle12)
    
    #Plot boundary layer thickness around spheres
    #OMEGA = 2.0*np.pi/PERIOD
    AMP_SMALL = RADIUSLARGE*0.8*0.8
    delta = np.sqrt(AMP_SMALL*RADIUSSMALL/float(Re))
    delta_norm = delta/RADIUSLARGE
    Circle5 = plt.Circle((0.0,0.5),1.0+delta_norm, clip_on=True,fill=False,edgecolor='orange')
    ax.add_artist(Circle5)
    Circle6 = plt.Circle((0.0,-2.0),0.5+delta_norm, clip_on=True,fill=False,edgecolor='orange')
    ax.add_artist(Circle6)
    Circle7 = plt.Circle((xU,yU),1.0+delta_norm, clip_on=True,fill=False,edgecolor='orange')
    ax.add_artist(Circle7)
    Circle8 = plt.Circle((xL,yL),0.5+delta_norm, clip_on=True,fill=False,edgecolor='orange')
    ax.add_artist(Circle8)
    
    return ax

Re = 25
allData = pd.read_csv(cwd_PYTHON+'/SimData/allSimData_Re{0}.csv'.format(Re),delimiter=' ')
HxValues = [4.25,5.75,3.75,2.75,1.25,0.75,0.25,0.25]
HyValues = [3.5,0.0,-0.5,-4.5,-3.5,-4.5,-4.5,-4.5]
ThetaValues = [337.5,90.0,337.5,112.5,112.5,90.0,90.0,112.5]
for jdx in range(len(HxValues)):
    if jdx == 1:
        sys.exit(0)
    Hx = HxValues[jdx]
    Hy = HyValues[jdx]
    Theta = ThetaValues[jdx]
    print('Hx = {0}: Hy = {1}: Theta = {2}'.format(Hx,Hy,Theta))
    thetaData = allData[allData['parThetaBW'] <= Theta + 0.01].copy()
    thetaData = thetaData[thetaData['parThetaBW'] >= Theta - 0.01].copy()
    HxData = thetaData[thetaData['parHx'] <= Hx + 0.01].copy()
    HxData = HxData[HxData['parHx'] >= Hx - 0.01].copy()
    HyData = HxData[HxData['parHy'] <= Hy + 0.01].copy()
    HyData = HyData[HyData['parHy'] >= Hy - 0.01].copy()
    HyData = HyData.sort_values(by=['time'])
    HyData = HyData.reset_index(drop=True)
    #print(HyData)
    #sys.exit(0)
    #Calculate out sphere placements
    HyData['nTx'] = np.cos(HyData['ThetaBW'] + np.pi/2.0)
    HyData['nTy'] = np.sin(HyData['ThetaBW'] + np.pi/2.0)
    HyData['bXU'] = HyData['Hx'] + 0.5*HyData['nTx']
    HyData['bYU'] = HyData['Hy'] + 0.5*HyData['nTy']
    HyData['bXL'] = HyData['Hx'] - 2.0*HyData['nTx']
    HyData['bYL'] = HyData['Hy'] - 2.0*HyData['nTy']
    print(len(HyData['time']))
    for idx in range(len(HyData['time'])):
        HyData.loc[idx,'time'] = np.round(HyData.loc[idx,'time'],1)
    #print(HyData['time'])
    for idxTime in range(len(HyData['time'])):
        time = np.round(PERIOD*idxTime,1)
        data = HyData[HyData['time'] == time].copy()
        #data = HyData[HyData['time'] <= time+0.01].copy()
        #data = data[data['time'] >= time - 0.01].copy()
        data = data.reset_index(drop=True)
        #print(data)
        assert len(data['time']) == 1
        #Get traj data
        trajData = HyData[HyData['time'] <= time].copy()
        trajData = trajData.reset_index(drop=True)
        #2D PHASE SPACE PLOTS (Hy vs Hx) Configurations and Dynamics
        #Figure and subplot for each Theta
        fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
        ax.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ vs. $H_x$'%(time,Theta),fontsize=15,**csfont)
        ax.set_xlabel(r'$H_x$',fontsize=12,**csfont)
        ax.set_ylabel(r'$H_y$',fontsize=12,**csfont)
        ax.axis([min(-1.0,np.amin(trajData['bXU'])-3.0),max(6.0,np.amax(trajData['bXU'])+3.0),
                 min(-8.0,np.amin(trajData['bYU'])-3.0),max(2.5,np.amax(trajData['bYU'])+3.0)])
        ax.set_aspect('equal')
        #Plot Swimmers and Traj Data
        ax = PlotSwimmers(data,trajData,Re,ax)
        #2D PLOTS
        fig.tight_layout()
        #plt.show()
        #sys.exit(0)
        strDir = cwd_PYTHON+"/../Figures/SwimFrameTrajectories/Re{0}/".format(Re)
        pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
        fig.savefig(strDir+'Traj_T{0}_Hx{1}_Hy{2}_{3}_.png'.format(Theta,Hx,Hy,idxTime))
        fig.clf()
        plt.close()
        if idxTime % 20 == 0:
            print('time = ',time)

    #Make movies
    strMovie = "ffmpeg -r 10 -i Traj_T{0}_Hx{1}_Hy{2}_%d_.png -vcodec libx264 -pix_fmt yuv420p -y TrajMov_T{0}_Hx{1}_Hy{2}_.mp4".format(Theta,Hx,Hy)
    os.chdir(cwd_PYTHON+"/../Figures/SwimFrameTrajectories/Re{0}/".format(Re))
    os.system(strMovie)
    fileName = "rm Traj_T{0}_Hx{1}_Hy{2}_*".format(Theta,Hx,Hy)
    os.system(fileName)
    os.chdir(cwd_PYTHON)


Hx = 4.25: Hy = 3.5: Theta = 337.5
157
time =  0.0
time =  2.0
time =  4.0
time =  6.0
time =  8.0
time =  10.0
time =  12.0
time =  14.0


SystemExit: 0

### Plot Last TimeStep For all Hy = 0 Sims

In [127]:
ReList = [2,7,10]

allViableConfigs = pd.read_csv(cwd_PYTHON+'/allViableConfigs.csv',delimiter=' ')
viableConfigs = allViableConfigs[allViableConfigs['Hy'] == 0.0].copy()
viableConfigs = viableConfigs.reset_index(drop=True)

for Re in ReList:
    allData = pd.read_csv(cwd_PYTHON+'/SimData/allSimData_Re{0}.csv'.format(Re),delimiter=' ')
    for idx in range(len(viableConfigs['Theta'])):
        Hx = np.round(viableConfigs.loc[idx,'Hx'],2)
        Hy = np.round(viableConfigs.loc[idx,'Hy'],1)
        Theta = np.round(viableConfigs.loc[idx,'Theta'],1)
        HxData = allData[allData['parHx'] == Hx]
        thetaData = HxData[HxData['parThetaBW'] == Theta]
        HyData = thetaData[thetaData['parHy'] == Hy]
        HyData = HyData.sort_values(by=['time']).reset_index(drop=True)
        #Calculate out sphere placements
        HyData['nTx'] = np.cos(HyData['ThetaBW'] + np.pi/2.0)
        HyData['nTy'] = np.sin(HyData['ThetaBW'] + np.pi/2.0)
        HyData['bXU'] = HyData['Hx'] + 0.5*HyData['nTx']
        HyData['bYU'] = HyData['Hy'] + 0.5*HyData['nTy']
        HyData['bXL'] = HyData['Hx'] - 2.0*HyData['nTx']
        HyData['bYL'] = HyData['Hy'] - 2.0*HyData['nTy']
        #Calculate change in Hx, Hy values
        HyData['vHx'],HyData['vHy'],HyData['vmag'] = 0.0,0.0,0.0
        HyData
        for jdx in range(len(HyData['time'])):
            HyData.loc[jdx,'time'] = np.round(HyData.loc[jdx,'time'],1)
        '''
        for jdx in range(1,len(HyData['time'])):
            #Calculate change in Hx, Hy values
            HyData.loc[jdx,'vHx'] = (HyData.loc[jdx,'Hx'] - HyData.loc[jdx-1,'Hx'])/PERIOD
            HyData.loc[jdx,'vHy'] = (HyData.loc[jdx,'Hy'] - HyData.loc[jdx-1,'Hy'])/PERIOD
            HyData.loc[jdx,'vmag'] = np.hypot(HyData.loc[jdx,'vHx'],HyData.loc[jdx,'vHy'])
            if HyData.loc[jdx,'vmag'] > L_norm/(4.0*PERIOD):
                #We have a periodic crossover
                #Remove data after crossover
                HyData = HyData[HyData['time'] < HyData.loc[jdx,'time']].copy()
                break
        '''
        data = HyData.tail(1)
        data = data.reset_index(drop=True)
        if len(data['time']) > 0:
            trajData = HyData.copy()
            #2D PHASE SPACE PLOTS (Hy vs Hx) Configurations and Dynamics
            #Figure and subplot for each Theta
            fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
            ax.set_title(r'time =%.1f: $\theta$ = %s: $H_y=%.1f$ vs. $H_x=%.2f$'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
            ax.set_xlabel(r'$H_x$',fontsize=12,**csfont)
            ax.set_ylabel(r'$H_y$',fontsize=12,**csfont)
            ax.axis([min(-1.0,np.amin(trajData['bXU'])-3.0),max(6.0,np.amax(trajData['bXU'])+3.0),
                     min(-8.0,np.amin(trajData['bYU'])-3.0),max(2.5,np.amax(trajData['bYU'])+3.0)])
            ax.set_aspect('equal')
            #Plot Swimmers and Traj Data
            print('Re = {0}: Hx = {1}: Hy = {2}: Theta = {3}'.format(Re,Hx,Hy,Theta))
            ax = PlotSwimmers(data,trajData,Re,ax)
            #2D PLOTS
            fig.tight_layout()
            #plt.show()
            #sys.exit(0)
            strDir = cwd_PYTHON+"/../Figures/FullTrajectory/Re{0}/Hx{1}/".format(Re,Hx)
            pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
            fig.savefig(strDir+'Traj_T{0}_Hx{1}_Hy{2}_.png'.format(Theta,Hx,Hy))
            fig.clf()
            plt.close()


Re = 2: Hx = 2.25: Hy = 0.0: Theta = 0.0
Re = 2: Hx = 2.25: Hy = 0.0: Theta = 180.0
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 0.0
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 22.5
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 90.0
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 157.5
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 180.0
Re = 2: Hx = 2.75: Hy = 0.0: Theta = 202.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 0.0
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 22.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 45.0
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 67.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 90.0
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 112.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 135.0
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 157.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 180.0
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 202.5
Re = 2: Hx = 3.25: Hy = 0.0: Theta = 337.5
Re = 2: Hx = 3.75: Hy = 0.0: Theta = 0.0
Re = 2: Hx = 3.75: Hy = 0.0: Theta = 22.5
Re = 2: Hx = 3.75: Hy = 0.0: Theta = 45.0
Re = 2: Hx = 3.75: Hy = 0.0: Theta = 67.5
Re = 2: Hx = 3.75: Hy = 0.0:

Re = 7: Hx = 5.25: Hy = 0.0: Theta = 270.0
Re = 7: Hx = 5.25: Hy = 0.0: Theta = 292.5
Re = 7: Hx = 5.25: Hy = 0.0: Theta = 315.0
Re = 7: Hx = 5.25: Hy = 0.0: Theta = 337.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 0.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 22.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 45.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 67.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 90.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 112.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 135.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 157.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 180.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 202.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 225.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 247.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 270.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 292.5
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 315.0
Re = 7: Hx = 5.75: Hy = 0.0: Theta = 337.5
Re = 7: Hx = 6.25: Hy = 0.0: Theta = 0.0
Re = 7: Hx = 6.25: Hy = 0.0: Theta = 22.5
Re = 7: Hx = 6.25: Hy = 0.0: Theta = 45.0
Re = 7: Hx = 6.25: Hy

### Plot Last Timestep for all Re=25 Sims

In [254]:
%matplotlib inline
ReList = ['25']

allViableConfigs = pd.read_csv(cwd_PYTHON+'/allViableConfigs.csv',delimiter=' ')
viableConfigs = allViableConfigs[allViableConfigs['Hx'] == 4.25].copy()
viableConfigs = viableConfigs.reset_index(drop=True)

for Re in ReList:
    allData = pd.read_csv(cwd_PYTHON+'/SimData/allSimData_Re{0}.csv'.format(Re),delimiter=' ')
    for idx in range(len(viableConfigs['Theta'])):
        Hx = np.round(viableConfigs.loc[idx,'Hx'],2)
        Hy = np.round(viableConfigs.loc[idx,'Hy'],1)
        Theta = np.round(viableConfigs.loc[idx,'Theta'],1)
        HxData = allData[allData['parHx'] == Hx]
        thetaData = HxData[HxData['parThetaBW'] == Theta]
        HyData = thetaData[thetaData['parHy'] == Hy]
        HyData = HyData.sort_values(by=['time']).reset_index(drop=True)
        #Calculate out sphere placements
        HyData['nTx'] = np.cos(HyData['ThetaBW'] + np.pi/2.0)
        HyData['nTy'] = np.sin(HyData['ThetaBW'] + np.pi/2.0)
        HyData['bXU'] = HyData['Hx'] + 0.5*HyData['nTx']
        HyData['bYU'] = HyData['Hy'] + 0.5*HyData['nTy']
        HyData['bXL'] = HyData['Hx'] - 2.0*HyData['nTx']
        HyData['bYL'] = HyData['Hy'] - 2.0*HyData['nTy']
        #Calculate change in Hx, Hy values
        for jdx in range(len(HyData['time'])):
            HyData.loc[jdx,'time'] = np.round(HyData.loc[jdx,'time'],1)
        data = HyData.tail(1)
        data = data.reset_index(drop=True)
        if len(data['time']) > 0:
            time = data.loc[0,'time']
            trajData = HyData.copy()
            #2D PHASE SPACE PLOTS (Hy vs Hx) Configurations and Dynamics
            #Figure and subplot for each Theta
            fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(6,6),dpi=250)
            ax.set_title(r'time =%.1f: $\theta$ = %s: $H_y=%.1f$ vs. $H_x=%.2f$'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
            ax.set_xlabel(r'$H_x$',fontsize=12,**csfont)
            ax.set_ylabel(r'$H_y$',fontsize=12,**csfont)
            ax.axis([min(-1.0,np.amin(trajData['bXU'])-3.0),max(6.0,np.amax(trajData['bXU'])+3.0),
                     min(-8.0,np.amin(trajData['bYU'])-3.0),max(2.5,np.amax(trajData['bYU'])+3.0)])
            ax.set_aspect('equal')
            #Plot Swimmers and Traj Data
            print('Re = {0}: Hx = {1}: Hy = {2}: Theta = {3}'.format(Re,Hx,Hy,Theta))
            print('Hx = %.2f: Hy = %.2f: Theta = %.2f: H = %.2f'%(data.loc[0,'Hx'],data.loc[0,'Hy'],180.0*data.loc[0,'ThetaBW']/np.pi,data.loc[0,'H']))
            ax = PlotSwimmers(data,trajData,Re,ax)
            #2D PLOTS
            fig.tight_layout()
            #plt.show()
            #sys.exit(0)
            strDir = cwd_PYTHON+"/../Figures/FullTrajectory/Re{0}/Hy{1}/".format(Re,Hy)
            pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
            fig.savefig(strDir+'Traj_T{0}_Hx{1}_Hy{2}_.png'.format(Theta,Hx,Hy))
            fig.clf()
            plt.close()


Re = 25: Hx = 4.25: Hy = -6.5: Theta = 0.0
Hx = 6.19: Hy = -12.30: Theta = 319.99: H = 13.77
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 22.5
Hx = 2.38: Hy = -1.50: Theta = 178.47: H = 2.81
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 45.0
Hx = 2.34: Hy = -1.59: Theta = 175.59: H = 2.83
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 67.5
Hx = 2.37: Hy = -1.46: Theta = 181.83: H = 2.78
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 90.0
Hx = 2.24: Hy = -1.71: Theta = 173.53: H = 2.82
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 112.5
Hx = 2.35: Hy = -1.48: Theta = 180.56: H = 2.78
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 135.0
Hx = 2.36: Hy = -1.47: Theta = 182.17: H = 2.78
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 157.5
Hx = 2.24: Hy = -1.67: Theta = 174.11: H = 2.80
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 180.0
Hx = 7.63: Hy = -31.24: Theta = 180.62: H = 32.16
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 202.5
Hx = -5.53: Hy = -27.31: Theta = 177.04: H = 27.86
Re = 25: Hx = 4.25: Hy = -6.5: Theta = 225.0
Hx = -23.67: Hy = 8.88: 

Re = 25: Hx = 4.25: Hy = -1.5: Theta = 202.5
Hx = 2.47: Hy = -1.41: Theta = 185.92: H = 2.84
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 225.0
Hx = 2.29: Hy = -1.69: Theta = 174.13: H = 2.85
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 247.5
Hx = 2.30: Hy = -1.58: Theta = 178.75: H = 2.79
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 270.0
Hx = 2.55: Hy = -0.36: Theta = 341.69: H = 2.58
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 292.5
Hx = 2.48: Hy = -0.50: Theta = 337.74: H = 2.53
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 315.0
Hx = 2.49: Hy = -0.46: Theta = 338.30: H = 2.54
Re = 25: Hx = 4.25: Hy = -1.5: Theta = 337.5
Hx = 2.50: Hy = -0.47: Theta = 336.38: H = 2.54
Re = 25: Hx = 4.25: Hy = -0.5: Theta = 0.0
Hx = 2.52: Hy = -0.35: Theta = 341.41: H = 2.55
Re = 25: Hx = 4.25: Hy = -0.5: Theta = 22.5
Hx = 2.53: Hy = -0.44: Theta = 341.00: H = 2.57
Re = 25: Hx = 4.25: Hy = -0.5: Theta = 45.0
Hx = 2.52: Hy = -0.48: Theta = 340.66: H = 2.57
Re = 25: Hx = 4.25: Hy = -0.5: Theta = 67.5
Hx = 2.46: Hy = -0.50: Theta =

Re = 25: Hx = 4.25: Hy = 3.5: Theta = 67.5
Hx = 2.52: Hy = -0.44: Theta = 341.33: H = 2.56
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 90.0
Hx = 2.46: Hy = -0.52: Theta = 336.66: H = 2.51
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 112.5
Hx = 0.08: Hy = 3.33: Theta = 177.34: H = 3.33
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 135.0
Hx = -0.09: Hy = 3.43: Theta = 183.99: H = 3.43
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 157.5
Hx = 0.33: Hy = 3.31: Theta = 169.51: H = 3.33
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 180.0
Hx = 2.41: Hy = -1.40: Theta = 182.96: H = 2.78
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 202.5
Hx = 2.36: Hy = -1.47: Theta = 181.70: H = 2.78
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 225.0
Hx = 2.35: Hy = -1.48: Theta = 181.56: H = 2.78
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 247.5
Hx = 2.38: Hy = -1.54: Theta = 181.50: H = 2.83
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 270.0
Hx = 2.47: Hy = -1.39: Theta = 185.78: H = 2.83
Re = 25: Hx = 4.25: Hy = 3.5: Theta = 292.5
Hx = 2.43: Hy = -1.45: Theta = 182.27: H

In [247]:
endData = pd.read_csv(cwd_PYTHON+'/SimData/allEndData_Re2_.csv',delimiter=' ')
LData = endData[endData['color'] == 'yellow']
LData = LData[LData['Hy'] > 0.0]
LData = LData[LData['Hx'] > 0.0]
LData = LData.reset_index(drop=True)
meanHx = np.mean(LData['Hx'])
meanHy = np.mean(LData['Hy'])
meanH = np.mean(LData['H'])
print('Hx = %.2f: Hy = %.2f: H = %.2f'%(meanHx,meanHy,meanH))
meanT = np.mean(LData['ThetaBW'])
print('T = %.2f'%(180.0*meanT/np.pi))

endData = pd.read_csv(cwd_PYTHON+'/SimData/allEndData_Re7_.csv',delimiter=' ')
LData = endData[endData['color'] == 'blue']
LData = LData[LData['Hx'] > 0.0]
LData = LData.reset_index(drop=True)
meanHx = np.mean(LData['Hx'])
meanHy = np.mean(LData['Hy'])
meanH = np.mean(LData['H'])
print('Hx = %.2f: Hy = %.2f: H = %.2f'%(meanHx,meanHy,meanH))
meanT = np.mean(LData['ThetaBW'])
print('T = %.2f'%(180.0*meanT/np.pi))


Hx = 3.94: Hy = 2.22: H = 4.52
T = 260.91
Hx = 3.12: Hy = -0.86: H = 3.24
T = 329.10
